In [1]:
# === Auracelle Charlie - Live 2025: Colab Setup (Reserved Domain) ===
# Installs, cleanup, and ngrok config for aiwargame.ngrok.app
import os

try:
    import pip
    !pip -q install --upgrade pip >/dev/null 2>&1
    !pip -q install streamlit pyngrok==7.2.1 plotly pyvis networkx geopandas folium shapely pyproj rtree >/dev/null 2>&1
except Exception as e:
    print("Dependency install encountered an issue (continuing):", e)

!pkill -f ngrok || true
!pkill -f streamlit || true

try:
    from pyngrok import ngrok
    try:
        ngrok.kill()
        ngrok.disconnect()
    except Exception:
        pass
except Exception:
    pass

os.environ["NGROK_AUTHTOKEN"] = "2vmh7uE9lpuOWrldBNSV68hJKH7_4Ukd3XG92jWofsVoZALiJ"
!ngrok config add-authtoken $NGROK_AUTHTOKEN

print("✅ Setup complete. Reserved domain: aiwargame.ngrok.app")

^C
^C
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
✅ Setup complete. Reserved domain: aiwargame.ngrok.app


# 🧬 Auracelle Charlie - AlphaFold-Style Influence Map

## Visualizing External Policy Pressures & Internal Cultural Forces

### 🎯 What This Shows

Unlike traditional network visualizations that show static relationships, this **AlphaFold-style influence map** reveals the **dynamic forces** acting on each country:

1. **External Policy Influences** (Arrows/Cones)
   - 🔴 Restrictive policies (US export controls, sanctions)
   - 🟠 Regulatory frameworks (EU AI Act, GDPR)
   - 🟡 Technical standards (competing frameworks)
   - 🔵 Governance initiatives (Arctic Council, UN)
   - Arrows point FROM policy source TO affected country
   - Arrow thickness = policy strength/impact

2. **Internal Cultural Forces** (Translucent Spheres)
   - 💙 Democratic accountability pressure
   - ❤️ Authoritarian control imperatives
   - 💚 Indigenous data sovereignty
   - 💜 Tech hub ambitions
   - Sphere size = force strength
   - Sphere encompasses all affected countries

3. **Country Positioning** (3D Space)
   - X-axis: Economic power (GDP)
   - Y-axis: Military capability (% GDP)
   - Z-axis: Tech infrastructure (Internet %)
   - Node size: Overall influence score
   - Node color: Cultural alignment

### 🔬 Key Insights

**Multiple arrows converging** → Country under high external pressure  
**Large encompassing sphere** → Strong internal cultural constraints  
**Arrows + Spheres overlapping** → Complex, constrained policy space  
**Isolated positioning** → Greater policy autonomy

### 📊 E-AGPO-HT Integration

This visualization operationalizes:
- **Stratum III (g-GWC)**: Rapid comprehension of multi-dimensional influence dynamics
- **Stratum II (SAD)**: Alliance detection through spatial clustering + shared force fields
- **Stratum II (STI)**: Tension identification via conflicting force vectors
- **Stratum I (NOF)**: Concrete policy pressures + cultural forces as operational factors

### 🚀 New Countries

- 🇬🇱 **Greenland**: Unique Arctic governance position, minimal external pressure
- 🇻🇪 **Venezuela**: High external pressure (sanctions) + strong internal forces

---

**Author**: Grace Evans | **Organization**: Auracelle AI Governance Labs  
**Framework**: E-AGPO-HT | **Affiliation**: UC Berkeley CLTC


In [2]:
# ========================================
# CELL 1: Install Dependencies
# ========================================
!pip install -q pyngrok streamlit networkx matplotlib numpy pandas torch plotly wbgapi requests scikit-learn

# ========================================
# CELL 2: Setup Environment & AGPO Data Package
# ========================================
import os, time, subprocess, sys
from pyngrok import ngrok
from IPython.display import display, HTML

# Cleanup existing processes
!pkill -f streamlit || true
!pkill -f ngrok || true
try:
    ngrok.disconnect("http://localhost:8501")
except:
    pass
try:
    ngrok.kill()
except:
    pass

ngrok.set_auth_token("2vmh7uE9lpuOWrldBNSV68hJKH7_4Ukd3XG92jWofsVoZALiJ")
os.makedirs("pages", exist_ok=True)
os.makedirs("agpo_data", exist_ok=True)
print("✅ Setup complete")

# ========================================
# CELL 3: Create AGPO Data Package - World Bank API Integration
# ========================================
agpo_worldbank = '''"""
AGPO Data Package - World Bank API Integration
Fetches real-world economic and development indicators
"""
import wbgapi as wb
import pandas as pd
import streamlit as st

@st.cache_data(ttl=3600)
def get_world_bank_indicator(indicator_code, country_codes, start_year=2015, end_year=2024):
    """
    Fetch World Bank indicator data for specified countries

    Args:
        indicator_code: WB indicator (e.g., 'NY.GDP.MKTP.CD' for GDP)
        country_codes: List of ISO3 country codes (e.g., ['USA', 'CHN'])
        start_year: Start year for data
        end_year: End year for data

    Returns:
        DataFrame with country, year, indicator value
    """
    try:
        data = wb.data.DataFrame(
            indicator_code,
            country_codes,
            time=range(start_year, end_year + 1),
            labels=True
        )

        # Reshape data
        data_reset = data.reset_index()
        data_melted = data_reset.melt(
            id_vars=['Country'],
            var_name='Year',
            value_name='Value'
        )
        data_melted['Indicator'] = indicator_code

        return data_melted
    except Exception as e:
        st.warning(f"World Bank API error: {e}")
        return pd.DataFrame()

@st.cache_data(ttl=3600)
def get_many_indicators(indicator_codes, country_codes=['USA', 'CHN', 'GBR', 'JPN', 'IND', 'BRA', 'ARE'], start_year=2015, end_year=2024):
    """
    Fetch multiple World Bank indicators at once

    Common indicators:
    - NY.GDP.MKTP.CD: GDP (current US$)
    - MS.MIL.XPND.GD.ZS: Military expenditure (% of GDP)
    - IT.NET.USER.ZS: Internet users (% of population)
    - SP.POP.TOTL: Total population
    - NY.GDP.PCAP.CD: GDP per capita
    """
    all_data = []
    for indicator in indicator_codes:
        df = get_world_bank_indicator(indicator, country_codes, start_year, end_year)
        if not df.empty:
            all_data.append(df)

    if all_data:
        return pd.concat(all_data, ignore_index=True)
    return pd.DataFrame()

def get_latest_gdp(country_code):
    """Get most recent GDP data for a country"""
    try:
        data = get_world_bank_indicator('NY.GDP.MKTP.CD', [country_code], 2020, 2024)
        if not data.empty:
            latest = data.dropna().sort_values('Year', ascending=False).iloc[0]
            value = latest['Value']
            # Handle both string and numeric values
            if isinstance(value, str):
                value = float(value)
            return value / 1e12  # Convert to trillions
        return None
    except Exception as e:
        return None

def get_latest_military_expenditure(country_code):
    """Get most recent military expenditure as % of GDP"""
    try:
        data = get_world_bank_indicator('MS.MIL.XPND.GD.ZS', [country_code], 2020, 2024)
        if not data.empty:
            latest = data.dropna().sort_values('Year', ascending=False).iloc[0]
            value = latest['Value']
            # Handle both string and numeric values
            if isinstance(value, str):
                value = float(value)
            return value
        return None
    except Exception as e:
        return None

def get_internet_penetration(country_code):
    """Get internet users as % of population"""
    try:
        data = get_world_bank_indicator('IT.NET.USER.ZS', [country_code], 2020, 2024)
        if not data.empty:
            latest = data.dropna().sort_values('Year', ascending=False).iloc[0]
            value = latest['Value']
            # Handle both string and numeric values
            if isinstance(value, str):
                value = float(value)
            return value
        return None
    except Exception as e:
        return None
'''

with open('agpo_data/worldbank.py', 'w') as f:
    f.write(agpo_worldbank)

# ========================================
# CELL 4: Create AGPO Data Package - Export Controls API
# ========================================
agpo_exportcontrol = '''"""
AGPO Data Package - US Export Controls Integration
Consolidated Screening List (CSL) from trade.gov
"""
import requests
import pandas as pd
import streamlit as st
from datetime import datetime

@st.cache_data(ttl=86400)  # Cache for 24 hours
def fetch_consolidated_screening_list():
    """
    Fetch US Consolidated Screening List (Entity List, DPL, UVL, etc.)
    Returns DataFrame with sanctioned entities
    Note: This API may require authentication or have rate limits
    """
    url = "https://api.trade.gov/consolidated_screening_list/search"

    try:
        params = {
            "size": 100,
            "offset": 0
        }

        response = requests.get(url, params=params, timeout=10)

        # Check response status
        if response.status_code == 403:
            # API requires authentication
            return pd.DataFrame()

        response.raise_for_status()

        # Check if response has content before parsing
        if not response.text or response.text.strip() == '':
            return pd.DataFrame()

        data = response.json()
        results = data.get('results', [])

        # Parse results
        records = []
        for item in results:
            records.append({
                'name': item.get('name', 'N/A'),
                'country': item.get('addresses', [{}])[0].get('country', 'N/A') if item.get('addresses') else 'N/A',
                'source': item.get('source', 'N/A'),
                'type': item.get('type', 'N/A'),
                'programs': ', '.join(item.get('programs', [])),
                'remarks': item.get('remarks', '')
            })

        return pd.DataFrame(records)

    except Exception as e:
        # Silently return empty DataFrame - API may be unavailable
        return pd.DataFrame()

def check_entity_sanctions(entity_name):
    """Check if an entity appears on screening lists"""
    df = fetch_consolidated_screening_list()
    if df.empty:
        return None

    matches = df[df['name'].str.contains(entity_name, case=False, na=False)]
    return matches

def get_sanctioned_countries():
    """Get list of countries with sanctioned entities"""
    df = fetch_consolidated_screening_list()
    if df.empty:
        return []

    return df['country'].value_counts().to_dict()
'''

with open('agpo_data/exportcontrol.py', 'w') as f:
    f.write(agpo_exportcontrol)

# ========================================
# CELL 5: Create AGPO Data Package - SIPRI Integration
# ========================================
agpo_sipri = '''"""
AGPO Data Package - SIPRI Military Expenditure Integration
Note: SIPRI requires manual CSV download. This module processes the data.
"""
import pandas as pd
import streamlit as st
import io

def parse_sipri_csv(uploaded_file):
    """
    Parse SIPRI military expenditure CSV
    Download from: https://www.sipri.org/databases/milex
    """
    try:
        df = pd.read_csv(uploaded_file, encoding='utf-8')
        return df
    except Exception as e:
        st.error(f"SIPRI CSV parsing error: {e}")
        return pd.DataFrame()

def get_military_expenditure_by_country(df, country_name, year=2023):
    """Extract military expenditure for specific country and year"""
    try:
        if 'Country' in df.columns and str(year) in df.columns:
            country_data = df[df['Country'] == country_name]
            if not country_data.empty:
                value = country_data[str(year)].iloc[0]
                return value
        return None
    except:
        return None

def get_top_military_spenders(df, year=2023, top_n=10):
    """Get top N military spenders for a given year"""
    try:
        if str(year) in df.columns:
            top = df.nlargest(top_n, str(year))[['Country', str(year)]]
            return top
        return pd.DataFrame()
    except:
        return pd.DataFrame()
'''

with open('agpo_data/sipri.py', 'w') as f:
    f.write(agpo_sipri)

# ========================================
# CELL 6: Create AGPO Package Init
# ========================================
agpo_init = '''"""
AGPO Data Package - AI Governance Policy Optimization
Real-world data integration for wargaming simulations
"""
from .worldbank import *
from .exportcontrol import *
from .sipri import *

__version__ = "1.0.0"
'''

with open('agpo_data/__init__.py', 'w') as f:
    f.write(agpo_init)

print("✅ AGPO Data Package created")

# ========================================
# CELL 7: Create Adjudicator Module with API Integration
# ========================================
adjudicator_code = '''import random
import numpy as np
from datetime import datetime
import sys
sys.path.insert(0, ".")

class AgenticAdjudicator:
    """AI Agentic Adjudicator - Neutral referee with real-world data integration"""

    def __init__(self, mode="neutral"):
        self.mode = mode
        self.event_history = []
        self.tension_index = 0.5
        self.shock_types = [
            "economic_sanctions", "cyber_attack", "public_protest",
            "un_resolution", "trade_disruption", "diplomatic_incident",
            "tech_breakthrough", "alliance_shift", "intel_leak"
        ]
        self.real_world_data = {}

    def integrate_real_world_data(self, country_code, gdp=None, mil_exp=None, internet=None, sanctions=None):
        """Store real-world data for adjudication calculations"""
        self.real_world_data[country_code] = {
            'gdp': gdp,
            'military_expenditure_pct': mil_exp,
            'internet_penetration': internet,
            'sanctioned_entities': sanctions
        }

    def calculate_tension_index(self, actor_positions, power_levels, alignment_graph):
        """Calculate geopolitical tension with real-world data weighting"""
        position_divergence = np.std([hash(p) % 100 for p in actor_positions.values()]) / 100
        power_imbalance = np.std(list(power_levels.values()))
        alignment_factor = 1.0 - (sum(alignment_graph.values()) / (len(alignment_graph) + 1e-6))

        # Add real-world military expenditure factor
        mil_exp_factor = 0.0
        if self.real_world_data:
            mil_exps = [d.get('military_expenditure_pct', 2.0) for d in self.real_world_data.values() if d.get('military_expenditure_pct')]
            if mil_exps:
                mil_exp_factor = (np.mean(mil_exps) - 2.0) / 10.0  # Normalize around 2% baseline

        tension = (position_divergence * 0.3 + power_imbalance * 0.2 + alignment_factor * 0.3 + abs(mil_exp_factor) * 0.2)
        self.tension_index = max(0.0, min(1.0, tension))
        return self.tension_index

    def detect_deception(self, stated_position, historical_actions, power_level):
        """Detect potential deception in actor statements"""
        deception_score = 0.0

        if len(historical_actions) > 0:
            consistency = sum([1 for a in historical_actions if a == stated_position]) / len(historical_actions)
            deception_score += (1.0 - consistency) * 0.5

        if power_level < 0.5 and "aggressive" in stated_position.lower():
            deception_score += 0.3

        return min(1.0, deception_score)

    def inject_shock(self, current_round, tension_level):
        """Inject external shock event based on tension and real-world factors"""
        # Increase shock probability if sanctions are detected
        sanctions_multiplier = 1.0
        if self.real_world_data:
            # Handle None values properly when summing sanctions
            total_sanctions = sum([d.get('sanctioned_entities', 0) for d in self.real_world_data.values() if d.get('sanctioned_entities') is not None])
            if total_sanctions > 0:
                sanctions_multiplier = 1.3

        shock_probability = tension_level * 0.3 * sanctions_multiplier

        if random.random() < shock_probability:
            shock_type = random.choice(self.shock_types)
            impact_magnitude = random.uniform(0.1, 0.5) * tension_level

            event = {
                "round": current_round,
                "type": shock_type,
                "magnitude": impact_magnitude,
                "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                "narrative": self._generate_narrative(shock_type, impact_magnitude),
                "real_world_triggered": sanctions_multiplier > 1.0
            }

            self.event_history.append(event)
            return event

        return None

    def _generate_narrative(self, shock_type, magnitude):
        """Generate narrative text for shock events"""
        severity = "severe" if magnitude > 0.3 else "moderate"
        narratives = {
            "economic_sanctions": f"Breaking: Coalition imposes {severity} economic sanctions",
            "cyber_attack": f"Alert: {severity.capitalize()} cyber incident targets critical infrastructure",
            "public_protest": f"Developing: {severity.capitalize()} public demonstrations challenge policy",
            "un_resolution": f"UN Security Council debates {severity} resolution",
            "trade_disruption": f"Economic shock: {severity.capitalize()} trade route disruption",
            "diplomatic_incident": f"Diplomatic crisis: {severity.capitalize()} incident strains relations",
            "tech_breakthrough": f"Tech update: {severity.capitalize()} AI capability announced",
            "alliance_shift": f"Geopolitical shift: {severity.capitalize()} realignment detected",
            "intel_leak": f"Intelligence alert: {severity.capitalize()} data leak exposed"
        }
        return narratives.get(shock_type, "Unknown event occurred")

    def adjudicate(self, actor_beliefs, power_levels, alignment_graph, current_round):
        """Main adjudication function with real-world data integration"""
        tension = self.calculate_tension_index(actor_beliefs, power_levels, alignment_graph)
        shock_event = self.inject_shock(current_round, tension)

        confidence = 1.0 - (tension * 0.3)

        deception_scores = {}
        for actor, belief in actor_beliefs.items():
            hist = [e.get(actor) for e in self.event_history if actor in e]
            deception_scores[actor] = self.detect_deception(belief, hist, power_levels.get(actor, 0.5))

        return {
            "tension_index": tension,
            "shock_event": shock_event,
            "confidence_score": confidence,
            "deception_scores": deception_scores,
            "narrative": shock_event["narrative"] if shock_event else "Situation stable",
            "round": current_round,
            "real_world_integrated": len(self.real_world_data) > 0
        }
'''

with open('adjudicator.py', 'w') as f:
    f.write(adjudicator_code)

print("✅ Enhanced Adjudicator with API integration created")

# ========================================
# CELL 8: Create Login Page
# ========================================
app_code = '''import streamlit as st

st.set_page_config(page_title="Auracelle Charlie 3 - War Gaming Stress-Testing Policy Governance Research Simulation/Prototype", layout="wide", initial_sidebar_state="collapsed")
st.title("🔐 Auracelle Charlie 3 - War Gaming Stress-Testing Policy Governance Research Simulation/Prototype")

if "authenticated" not in st.session_state:
    st.session_state["authenticated"] = False

with st.form("login_form"):
    username = st.text_input("Username")
    password = st.text_input("Password", type="password")

    st.markdown("### 🎮 Phase 3 Features and Functionality")

    st.markdown("**Capabilities**")

    capabilities = [
        "🌍 World Bank API (GDP, military expenditure, internet penetration)",
        "🚫 US Export Controls API (sanctions screening)",
        "💥 External shock injection system",
        "🎭 Deception detection with real-world data",
        "🗺️ 3-D Influence Map",
        "🛡️ Red Teaming Foresight",
        "🧠 Evans-AGPO-HT Cognitive Science Framework"
    ]

    for c in capabilities:
        st.write(c)


    submit = st.form_submit_button("Login")

if submit:
    if password == "charlie2025":
        st.session_state["authenticated"] = True
        st.session_state["username"] = username
        st.switch_page("pages/2_Simulation.py")
    else:
        st.error("Incorrect password. Access denied.")

if st.session_state.get("authenticated", False):
    st.switch_page("pages/2_Simulation.py")
'''

with open('app.py', 'w') as f:
    f.write(app_code)

print("✅ Login page created")

# ========================================
# CELL 9: Create Simulation Page with Vertical Layout
# ========================================
sim_code = '''import streamlit as st
import streamlit.components.v1 as components

st.set_page_config(page_title="Auracelle Charlie 3", layout="wide", initial_sidebar_state="expanded")

if not st.session_state.get("authenticated", False):
    st.warning("⚠️ Please log in first.")
    st.switch_page("app.py")

# Load vertical simulation HTML
with open("simulation_vertical.html", "r", encoding="utf-8") as f:
    html_content = f.read()

components.html(html_content, height=2800, scrolling=True)
'''

with open('pages/2_Simulation.py', 'w') as f:
    f.write(sim_code)

print("✅ Simulation page with vertical layout created")

# ========================================
# CELL 11: Add pages: Real-World Data Metrics + INSTRUCTIONS
# ========================================
real_world_metrics_code = r"""import streamlit as st
import pandas as pd
import plotly.express as px

st.set_page_config(page_title="Real-World Data Metrics", page_icon="🌍", layout="wide", initial_sidebar_state="collapsed")

# Auth gate (same pattern as the app)
if not st.session_state.get("authenticated", False):
    st.warning("Please log in first.")
    st.switch_page("app.py")

st.header("🌍 Real-World Data")

# Actor selector
actor_options = ["Dubai","United Kingdom","United States","Japan","China","Brazil","India","NATO"]
default_a = st.session_state.get("selected_country_a", "United Kingdom")
default_b = st.session_state.get("selected_country_b", "United States")

col1, col2 = st.columns(2)
with col1:
    selected_actor_a = st.selectbox("Actor A", actor_options, index=actor_options.index(default_a) if default_a in actor_options else 1)
with col2:
    selected_actor_b = st.selectbox("Actor B", actor_options, index=actor_options.index(default_b) if default_b in actor_options else 2)

st.session_state["selected_country_a"] = selected_actor_a
st.session_state["selected_country_b"] = selected_actor_b

# Mappings (WB uses ISO3; NATO is not a WB country entity)
ACTOR_TO_ISO3 = {
    "Dubai": "ARE",              # UAE proxy (as in Charlie map)
    "United Kingdom": "GBR",
    "United States": "USA",
    "Japan": "JPN",
    "China": "CHN",
    "Brazil": "BRA",
    "India": "IND",
}

INDICATORS = {
    "gdp": ("NY.GDP.MKTP.CD", "GDP (current US$)"),
    "mil_exp": ("MS.MIL.XPND.GD.ZS", "Military expenditure (% of GDP)"),
    "internet": ("IT.NET.USER.ZS", "Internet users (% of population)"),
}

# Install and import World Bank API
AGPO_AVAILABLE = True
try:
    import wbgapi as wb
except ImportError:
    try:
        import subprocess
        import sys
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "wbgapi"])
        import wbgapi as wb
        st.success("✅ World Bank API installed successfully")
    except Exception as e:
        AGPO_AVAILABLE = False
        st.error(f"Could not install World Bank API: {e}")
except Exception as e:
    AGPO_AVAILABLE = False
    st.error(f"World Bank API error: {e}")

tabs = st.tabs(["📊 Economic Indicators", "🚫 Export Controls", "📜 Event History", "🧪 Batch Evaluation"])

with tabs[0]:
    st.subheader("📊 Economic Indicators")
    if not AGPO_AVAILABLE:
        st.error("World Bank client not available in this environment.")
    else:
        iso3_list = [ACTOR_TO_ISO3.get(a) for a in [selected_actor_a, selected_actor_b] if ACTOR_TO_ISO3.get(a)]
        label_list = [a for a in [selected_actor_a, selected_actor_b] if ACTOR_TO_ISO3.get(a)]

        if not iso3_list:
            st.info("Selected actors are not available in World Bank country data (e.g., NATO).")
        else:
            start_year, end_year = 2015, 2024

            def fetch_latest(ind_code: str) -> pd.DataFrame:
                try:
                    df = wb.data.DataFrame(ind_code, iso3_list, time=range(start_year, end_year + 1), labels=True)
                    # df index: Country; columns: years (as strings) -> values
                    df = df.reset_index().rename(columns={"economy": "iso3", "Country": "country"})
                    # Keep only year columns
                    year_cols = [c for c in df.columns if str(c).isdigit()]
                    # Find latest non-null per row
                    def latest_nonnull(row):
                        for y in sorted(year_cols, key=lambda x: int(x), reverse=True):
                            v = row.get(y)
                            if pd.notna(v):
                                return float(v), int(y)
                        return None, None

                    latest_vals = df.apply(lambda r: latest_nonnull(r), axis=1, result_type="expand")
                    df["value"], df["year"] = latest_vals[0], latest_vals[1]
                    return df[["country", "value", "year"]]
                except Exception as e:
                    st.warning(f"World Bank API error: {e}")
                    return pd.DataFrame(columns=["country", "value", "year"])

            rows = []
            for key, (code, label) in INDICATORS.items():
                t = fetch_latest(code)
                if not t.empty:
                    t["metric"] = key
                    t["label"] = label
                    rows.append(t)

            if not rows:
                st.warning("No indicator data returned for the selected actors.")
            else:
                long = pd.concat(rows, ignore_index=True)
                wide = long.pivot_table(index="country", columns="metric", values="value", aggfunc="first")
                st.dataframe(wide, use_container_width=True)

                # Plots
                if "gdp" in wide.columns:
                    fig_gdp = px.bar(wide.reset_index(), x="country", y="gdp", title="GDP Comparison (latest available year)")
                    st.plotly_chart(fig_gdp, use_container_width=True)

                if "mil_exp" in wide.columns:
                    fig_mil = px.bar(wide.reset_index(), x="country", y="mil_exp", title="Military Expenditure (% of GDP) (latest available year)")
                    st.plotly_chart(fig_mil, use_container_width=True)

                st.markdown("#### 🌐 Internet Penetration")
                if "internet" in wide.columns:
                    fig_int = px.bar(wide.reset_index(), x="country", y="internet", title="Internet Users (% of population) (latest available year)")
                    st.plotly_chart(fig_int, use_container_width=True)
                else:
                    st.info("Internet penetration data not available for the selected actors.")

with tabs[1]:
    st.subheader("🚫 Export Controls & Sanctions Screening")

    # US Export Controls - BIS Entity List screening
    st.markdown("### 🇺🇸 U.S. Bureau of Industry and Security (BIS) - Entity List")

    # Map actors to potential entities
    entity_search_terms = {
        "China": ["Huawei", "SMIC", "YMTC"],
        "Dubai": ["UAE entities"],
        "United Kingdom": ["No major entities"],
        "United States": ["No entities listed"],
        "Japan": ["No major entities"],
        "Brazil": ["No major entities"],
        "India": ["No major entities"],
        "NATO": ["N/A - Alliance"]
    }

    col1, col2 = st.columns(2)

    with col1:
        st.markdown(f"**{selected_actor_a}**")
        terms = entity_search_terms.get(selected_actor_a, ["No data"])
        if terms[0] == "No major entities" or terms[0] == "No entities listed":
            st.success(f"✅ {terms[0]} on BIS Entity List")
        elif terms[0] == "N/A - Alliance":
            st.info("N/A - Alliance entity")
        else:
            st.warning(f"⚠️ Known entities: {', '.join(terms)}")
            st.caption("Note: Presence on Entity List restricts technology exports")

    with col2:
        st.markdown(f"**{selected_actor_b}**")
        terms = entity_search_terms.get(selected_actor_b, ["No data"])
        if terms[0] == "No major entities" or terms[0] == "No entities listed":
            st.success(f"✅ {terms[0]} on BIS Entity List")
        elif terms[0] == "N/A - Alliance":
            st.info("N/A - Alliance entity")
        else:
            st.warning(f"⚠️ Known entities: {', '.join(terms)}")
            st.caption("Note: Presence on Entity List restricts technology exports")

    st.markdown("---")
    st.markdown("### 🌐 International Sanctions Regimes")
    st.info("💡 **Integration Note**: This section can be enhanced with live API connections to:")
    st.markdown("- **OFAC Sanctions List** (U.S. Treasury)")
    st.markdown("- **EU Sanctions Database**")
    st.markdown("- **UN Security Council Sanctions**")
    st.markdown("- **UK FCDO Sanctions List**")
    st.markdown("")
    st.markdown("These APIs provide real-time screening against sanctioned entities, individuals, and jurisdictions.")

with tabs[2]:
    st.subheader("📜 Event History")
    st.info("Placeholder: show recent shocks, policy actions, and adjudication trace entries.")

with tabs[3]:
    st.subheader("🧪 Batch Evaluation")
    st.info("Placeholder: run batch scenarios/policies/actors for comparison. (Hook point for your evaluation runner.)")"""
with open("pages/21_Real_World_Data_Metrics.py", "w") as f:
    f.write(real_world_metrics_code)

instructions_code = r"""import streamlit as st

st.set_page_config(page_title="INSTRUCTIONS and COGNITIVE SCIENCE MECHANICS", page_icon="📘", layout="wide", initial_sidebar_state="collapsed")

if not st.session_state.get("authenticated", False):
    st.warning("Please log in first.")
    st.switch_page("app.py")

st.header("📘 INSTRUCTIONS and COGNITIVE SCIENCE MECHANICS")

with st.expander("📘 Phase 3: Real-World Data-Driven Simulation Mechanics (Behind the Scenes)", expanded=True):
    st.markdown('\n### Phase 3: Real-World Data-Driven Simulation Mechanics (Behind the Scenes)\n\nAuracelle Charlie uses the **Evans-AGPO-HT** hierarchical theory as a *cognitive decision-science scaffold* to translate\nreal-world evidence streams (World Bank, export controls/sanctions screening, SIPRI uploads) into **stress-testing signals**\nfor policy governance decisions.\n\n#### 1) Hierarchical factor model (three strata)\nObserved governance performance is modeled as a hierarchical linear structure:\n\n- **Y** = observed governance performance on a task (per actor, round, and capability)\n- **g-GWC** = general governance wargaming capacity (Stratum III)\n- **BGC** = broad governance capability (Stratum II; 7 capabilities)\n- **NOF** = narrow operational factor (Stratum I; measurable sub-factors)\n- **Λ** = factor loadings; **ε** = measurement error\n\nConceptually:\n- Stratum III (**g-GWC**) explains overall “wargaming capacity”.\n- Stratum II (**BGC**) decomposes that capacity into broad competencies.\n- Stratum I (**NOF**) ties each competency to measurable operational signals (including real-world data).\n\n#### 2) The seven Broad Governance Capabilities (BGC)\nThe framework defines seven broad capabilities (examples of sub-constructs shown):\n- **STI** (Strategic Threat Intelligence): PIP, GRD, APM, TLA, PDI, AIP  \n- **SAD** (Security Architecture Design): TPD, MPM, NBE, DCF, CNS  \n- **ESI** (Exploratory Simulation Intelligence): DRS, CD, WIS, TRS, AGM, DBP  \n- **NDM** (Negotiation Dynamics Modeling): ABN, MTD, TMA, CFD, CMD, MCC  \n- **SRA** (Strategic Rationality Assessment): TAC, RLA, TPI, PRM, LTR  \n- **IIC** (Institutional Implementation Capacity): BTA, AMD, CTM, ELC, NES  \n- **ASI** (Adaptive Scalability Intelligence): CGA, RAC, VMD, KCS, FLI, CLP\n\n#### 3) Aggregation of capability into g-GWC\ng-GWC is computed as a weighted integration of BGCs (optionally multiplicative for synergy), and can include\nAI-acceleration factors (e.g., computational simulation speed, neural network integration, RL optimization).\n\n#### 4) Network effects in multi-agent governance\nInfluence propagation is modeled over a governance influence network:\n- next_state ≈ (1 − α)·current_state + α·W·current_state + interventions  \nwhere **W** is a weighted adjacency matrix (influence edges). This connects directly to the **3-D Influence Map**.\n\n#### 5) How this becomes “cognitive decision science” in Charlie\nIn Charlie, the math is used to:\n- **Explain** why an actor’s position is plausible given evidence (traceability).\n- **Adjudicate** between competing claims (agentic adjudicator + deception signals).\n- **Stress-test** policies across rounds by injecting shocks and measuring capability shifts and convergence/divergence.\n')

with st.expander("📘 Phase 3 Instructions - Player Walkthrough (Policy Stress-Testing War Game)", expanded=True):
    st.markdown('\n### Phase 3 Instructions - Player Walkthrough (Policy Stress-Testing War Game)\n\n1. **Select policy instrument to stress-test** (e.g., AI Act / GDPR variants).  \n2. **Pick two actors** and the **role lenses** you want to compare (institutional incentives).  \n3. **Choose who you represent** (your decision lens).  \n4. **Run rounds**: each round is a decision cycle—positions, adjudication, deception detection, and evidence-linked impacts.  \n5. **Observe outputs**:\n   - **Agentic adjudicator status**: neutral synthesis of signals, flags, and disagreements.\n   - **Deception detection**: consistency checks vs. historical actions, power/incentive mismatch, and evidence divergence.\n   - **Policy position comparison**: side-by-side evidence-linked posture and capability context.\n   - **Strategic analysis**: why the sandbox recommends a posture and what assumptions drive it.\n6. **Iterate**: adjust assumptions, inject shocks, and compare trajectories across actors and roles.\n')
"""
with open("pages/91_INSTRUCTIONS_and_COGNITIVE_SCIENCE_MECHANICS.py", "w") as f:
    f.write(instructions_code)

print("✅ Added pages: Real-World Data Metrics + INSTRUCTIONS")

# ========================================
# CELL 10: Create Red Team Module Page (Sidebar-clickable)
# ========================================
red_team_code = '''import streamlit as st
import pandas as pd
import numpy as np
import json

st.set_page_config(page_title="Red Team Module", layout="wide")

st.title("🛡️ Red Team Module — Foresight Cognition & Belief Distortion")
st.caption("Stress-test how actors perceive, misperceive, and act on futures by attacking signals, framing, or cognition parameters.")

# -----------------------------
# Helpers
# -----------------------------
def clip01(x: float) -> float:
    return float(max(0.0, min(1.0, x)))

def init_agents(names):
    agents = {}
    for n in names:
        agents[n] = {
            "cognition": {
                "H": 3,          # Horizon depth (1..K)
                "Omega": 0.55,   # Openness/update rate (0..1)
                "Lambda": 0.55,  # Uncertainty tolerance (0..1)
                "Pi": 0.45       # Narrative lock-in (0..1)
            },
            "belief": {
                "mu": 0.50,      # Expected outcome proxy (0..1)
                "sigma": 0.25    # Uncertainty proxy (0..1)
            },
            "metrics": {
                "USI": None,     # Update suppression index
                "HD": 0          # Horizon degradation
            }
        }
    return agents

def compute_alpha(cog):
    # alpha = Omega * (1 - Pi)
    return clip01(cog["Omega"] * (1.0 - cog["Pi"]))

def update_belief(agent, evidence):
    """
    Cognition-weighted belief update (lightweight, explainable).
    evidence: float in [-1, 1] where + pushes mu upward, - downward
    """
    cog = agent["cognition"]
    bel = agent["belief"]

    alpha = compute_alpha(cog)

    # Map evidence to [0,1] target for mu
    target = clip01(0.5 + 0.5 * float(evidence))

    bel["mu"] = clip01((1 - alpha) * bel["mu"] + alpha * target)

    # Uncertainty update: low Lambda collapses uncertainty faster; high Lambda stays more agnostic
    collapse = (1.0 - cog["Lambda"]) * 0.20
    bel["sigma"] = clip01(bel["sigma"] * (1.0 - collapse))

    # Metrics
    agent["metrics"]["USI"] = clip01(1.0 - alpha)

def apply_red_team_move(agent, move, intensity, K=5):
    cog = agent["cognition"]
    bel = agent["belief"]

    if move == "Horizon Collapse":
        old = cog["H"]
        cog["H"] = int(max(1, cog["H"] - int(round(intensity * 2))))
        agent["metrics"]["HD"] += (old - cog["H"])

    elif move == "Narrative Entrenchment":
        cog["Pi"] = clip01(cog["Pi"] + 0.35 * intensity)

    elif move == "Epistemic Distrust":
        cog["Omega"] = clip01(cog["Omega"] - 0.35 * intensity)

    elif move == "Panic Amplification":
        cog["Lambda"] = clip01(cog["Lambda"] - 0.35 * intensity)

    elif move == "Metric Spoofing":
        # Push mu in a misleading direction without improving the agent's cognition (signal tampering)
        bel["mu"] = clip01(bel["mu"] + (0.25 * intensity))

    elif move == "Frame Flip":
        # Invert the meaning of recent evidence by effectively flipping openness for one step
        cog["Omega"] = clip01(1.0 - cog["Omega"])

    else:
        pass

# -----------------------------
# State bootstrap
# -----------------------------
# Try to reuse existing Simulation state if present; otherwise initialize safely
if "charlie_agents" not in st.session_state:
    # Prefer any actor list from Simulation (if you stored it), else default to your Charlie set
    default_actors = st.session_state.get("actor_list") or ["Dubai", "UK", "US", "Japan", "China", "Brazil", "India", "NATO"]
    st.session_state["charlie_agents"] = init_agents(default_actors)

agents = st.session_state["charlie_agents"]
actor_names = list(agents.keys())

# -----------------------------
# Sidebar controls (clickable module config)
# -----------------------------
st.sidebar.header("Red Team Controls")
target = st.sidebar.selectbox("Target actor", actor_names, index=0)
move = st.sidebar.selectbox(
    "Belief-distortion move",
    [
        "Narrative Entrenchment",
        "Epistemic Distrust",
        "Horizon Collapse",
        "Panic Amplification",
        "Metric Spoofing",
        "Frame Flip",
    ],
    index=0
)
intensity = st.sidebar.slider("Intensity", 0.0, 1.0, 0.5, 0.05)

evidence = st.sidebar.slider("New evidence signal (for belief update)", -1.0, 1.0, 0.2, 0.05)
apply_move = st.sidebar.button("Apply Red Team Move", use_container_width=True)
apply_update = st.sidebar.button("Run Cognition-Weighted Belief Update", use_container_width=True)
reset = st.sidebar.button("Reset module state", use_container_width=True)

if reset:
    st.session_state["charlie_agents"] = init_agents(actor_names)
    agents = st.session_state["charlie_agents"]
    st.success("Reset Red Team module state.")

# -----------------------------
# Execute actions
# -----------------------------
if apply_move:
    apply_red_team_move(agents[target], move, intensity)
    st.success(f"Applied **{move}** to **{target}** (intensity={intensity:.2f}).")

if apply_update:
    update_belief(agents[target], evidence)
    st.success(f"Updated beliefs for **{target}** using evidence={evidence:.2f}.")

# -----------------------------
# Display
# -----------------------------
st.markdown(
    """
<style>
/* reduce top padding and tighten layout a bit */
section.main > div { padding-top: 1rem; }
/* tighten spacing above the dataframe */
div[data-testid="stDataFrame"] { margin-top: 0.25rem; }
</style>
""",
    unsafe_allow_html=True,
)

col1, col2 = st.columns([1, 1], gap="large")

with col1:
    st.subheader("Target actor — cognition & belief")
    st.write(f"**Actor:** {target}")

    cog = agents[target]["cognition"]
    bel = agents[target]["belief"]
    met = agents[target]["metrics"]

    st.markdown("**Cognition state (C) — real-world fields**")
    st.markdown("""
- **H** (Planning Horizon): how many turns ahead the actor considers
- **Omega** (Update Openness): willingness to revise beliefs when new evidence arrives
- **Lambda** (Uncertainty Tolerance): comfort operating under ambiguity and incomplete information
- **Pi** (Narrative Commitment): how strongly the actor is locked into a doctrine/storyline
""")
    st.code(json.dumps(cog, indent=2), language="json")

    st.markdown("**Belief state (b) — real-world fields**")
    st.markdown("""
- **mu** (Expected Outcome Score): 0.0 to 1.0 proxy for how well things are going
- **sigma** (Uncertainty Level): 0.0 to 1.0 proxy for how unsure the actor is
""")
    st.code(json.dumps(bel, indent=2), language="json")

    st.markdown("**Cognition metrics**")
    st.markdown("""
- **USI** (Update Suppression Index): higher means less learning / more "stuck"
- **HD** (Horizon Degradation): how much the planning horizon has been reduced by attacks
""")
    st.code(json.dumps(met, indent=2), language="json")

with col2:
    st.subheader("Data & evidence provenance (what feeds the sandbox)")
    st.info(
        "Auracelle Charlie is designed to be data-driven and evidence-based. "
        "This Red Team module manipulates cognition variables, but it is intended to sit on top of the "
        "real-world data streams used in the Simulation page."
    )

    st.markdown("**Current data resources (in this baseline):**")
    st.markdown("""
- **World Bank (wbgapi)**: macro indicators (GDP, population, etc.) pulled via API
- **U.S. Consolidated Screening List (CSL)**: export-control / sanctions screening via API wrapper
- **SIPRI**: military expenditure reference data (CSV upload)

For assurance, add an Evidence Ledger export that records each metric with dataset name, vintage/date, access method, and transformation.
""")

    st.markdown("**Recommended NATO-grade traceability upgrades:**")
    st.markdown("""
- Attach a **Source** tag to every metric (dataset name, vintage/date, access method)
- Attach a **Transformation** tag (how the raw data is normalized/scored)
- Provide an **Export** button for an **Evidence Ledger** (metric → source → transformation → timestamp/round)
""")

st.subheader("All actors — cognition & belief table")
rows = []
for name, a in agents.items():
    c = a["cognition"]
    b = a["belief"]
    m = a["metrics"]
    alpha = compute_alpha(c)
    rows.append({
        "Actor": name,
        "Planning horizon (H)": c["H"],
        "Evidence update openness (Omega)": round(c["Omega"], 3),
        "Uncertainty tolerance (Lambda)": round(c["Lambda"], 3),
        "Narrative lock-in (Pi)": round(c["Pi"], 3),
        "Learning rate (alpha)": round(alpha, 3),
        "Expected outcome (mu)": round(b["mu"], 3),
        "Uncertainty (sigma)": round(b["sigma"], 3),
        "Update suppression (USI)": None if m["USI"] is None else round(m["USI"], 3),
        "Horizon degradation (HD)": m["HD"],
    })
df = pd.DataFrame(rows).sort_values("Actor")
st.dataframe(df, use_container_width=True, hide_index=True)

st.divider()
st.markdown(
    """
**How to use this module**
- Use **Apply Red Team Move** to distort cognition (H, Omega, Lambda, Pi) or distort signals (Metric Spoofing / Frame Flip).
- Use **Run Cognition-Weighted Belief Update** to see how the target updates beliefs under its current cognition state.
- The key diagnostic is **USI (Update Suppression Index)**: higher = more “stuck” / less learning.
"""
)
'''

with open('pages/3_Red_Team_Module.py', 'w') as f:
    f.write(red_team_code)

print("✅ Full simulation page with API integration created")

# ========================================
# CELL 10: Launch Application
# ========================================
streamlit_process = subprocess.Popen(
    ["streamlit", "run", "app.py", "--server.port", "8501", "--server.address", "0.0.0.0"]
)
time.sleep(5)

ngrok_process = subprocess.Popen(
    ["ngrok", "http", "--domain=julene-untaxable-raelene.ngrok-free.dev", "8501"]
)

public_url = "https://aiwargame.ngrok.app"
print(f"\n🚀 Auracelle Charlie 3 - War Gaming Stress-Testing Policy Governance Research Simulation/Prototype with Full API Integration is LIVE!")
print(f"🔗 Access at: {public_url}")
print(f"\n✨ Features:")
print("  • 🤖 AI Agentic Adjudicator")
print("  • 🌍 World Bank API (GDP, Military, Internet)")
print("  • 🚫 US Export Controls API (Sanctions)")
print("  • 💥 External Shock System")
print("  • 🎭 Deception Detection")
print("  • 📊 Real-World Data Dashboard")
print("  • 📈 Economic & Military Analysis")
print("  • 📥 Export Reports (Markdown & CSV)")
print(f"\n🔑 Password: charlie2025")
print(f"\n📊 Data Sources:")
print("  - World Bank: Automatic")
print("  - Export Controls: Automatic")
print("  - SIPRI: Upload CSV in sidebar")

display(HTML(f'<a href="{public_url}" target="_blank" style="font-size:20px; font-weight:bold; color:#0066cc;">🔗 Launch Auracelle Charlie - Live 2025</a>'))

^C
^C
✅ Setup complete
✅ AGPO Data Package created
✅ Enhanced Adjudicator with API integration created
✅ Login page created
✅ Simulation page with vertical layout created
✅ Added pages: Real-World Data Metrics + INSTRUCTIONS
✅ Full simulation page with API integration created

🚀 Auracelle Charlie 3 - War Gaming Stress-Testing Policy Governance Research Simulation/Prototype with Full API Integration is LIVE!
🔗 Access at: https://aiwargame.ngrok.app

✨ Features:
  • 🤖 AI Agentic Adjudicator
  • 🌍 World Bank API (GDP, Military, Internet)
  • 🚫 US Export Controls API (Sanctions)
  • 💥 External Shock System
  • 🎭 Deception Detection
  • 📊 Real-World Data Dashboard
  • 📈 Economic & Military Analysis
  • 📥 Export Reports (Markdown & CSV)

🔑 Password: charlie2025

📊 Data Sources:
  - World Bank: Automatic
  - Export Controls: Automatic
  - SIPRI: Upload CSV in sidebar


In [3]:
# Write simulation HTML file
html_content = '''<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Auracelle Charlie 3 - War Gaming Simulation</title>
    <style>
        * {
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }

        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            background: #0f1419;
            color: #e8eaed;
            overflow-x: hidden;
        }

        .simulation-container {
            display: grid;
            grid-template-rows: 70px 1fr;
            height: 100vh;
            background: linear-gradient(135deg, #0f1419 0%, #1a1f2e 100%);
        }

        .header-bar {
            background: linear-gradient(180deg, #1e2530 0%, #161b24 100%);
            border-bottom: 2px solid #2d3748;
            display: flex;
            justify-content: space-between;
            align-items: center;
            padding: 0 25px;
            box-shadow: 0 2px 10px rgba(0,0,0,0.5);
            z-index: 100;
        }

        .platform-title {
            display: flex;
            align-items: center;
            gap: 15px;
        }

        .platform-icon {
            font-size: 2rem;
        }

        .title-text h1 {
            font-size: 1.4rem;
            font-weight: 600;
            color: #00d4ff;
            line-height: 1.2;
        }

        .title-text p {
            font-size: 0.8rem;
            color: #9ca3af;
            margin-top: 2px;
        }

        .header-actions {
            display: flex;
            gap: 15px;
            align-items: center;
        }

        .scenario-badge {
            padding: 8px 16px;
            background: rgba(0, 212, 255, 0.1);
            border: 1px solid #00d4ff;
            border-radius: 6px;
            font-size: 0.85rem;
            color: #00d4ff;
            font-weight: 600;
            cursor: pointer;
            transition: all 0.2s;
        }

        .scenario-badge:hover {
            background: rgba(0, 212, 255, 0.2);
            box-shadow: 0 0 15px rgba(0, 212, 255, 0.3);
        }

        .status-badge {
            display: flex;
            align-items: center;
            gap: 8px;
            padding: 6px 14px;
            background: rgba(16, 185, 129, 0.15);
            border: 1px solid #10b981;
            border-radius: 6px;
            font-size: 0.85rem;
        }

        .status-dot {
            width: 8px;
            height: 8px;
            background: #10b981;
            border-radius: 50%;
            animation: pulse-dot 2s infinite;
        }

        @keyframes pulse-dot {
            0%, 100% { opacity: 1; box-shadow: 0 0 8px #10b981; }
            50% { opacity: 0.6; box-shadow: 0 0 4px #10b981; }
        }

        .dashboard-grid {
            display: grid;
            grid-template-columns: 1fr;
            gap: 0;
            overflow: visible;
            height: auto;
        }

        .config-panel {
            background: #161b24;
            border-right: 1px solid #2d3748;
            overflow-y: auto;
            box-shadow: 2px 0 10px rgba(0,0,0,0.3);
        }

        .config-section {
            border-bottom: 1px solid #2d3748;
            padding: 24px;
        }

        .section-header {
            font-size: 0.75rem;
            text-transform: uppercase;
            letter-spacing: 0.1em;
            color: #00d4ff;
            margin-bottom: 18px;
            font-weight: 700;
            display: flex;
            align-items: center;
            gap: 8px;
        }

        .scenario-card {
            background: linear-gradient(135deg, rgba(0, 212, 255, 0.08) 0%, rgba(0, 123, 255, 0.04) 100%);
            border: 2px solid #00d4ff;
            border-radius: 12px;
            padding: 18px;
            margin-bottom: 16px;
            cursor: pointer;
            transition: all 0.3s;
        }

        .scenario-card:hover {
            transform: translateY(-2px);
            box-shadow: 0 8px 20px rgba(0, 212, 255, 0.2);
        }

        .scenario-title {
            font-size: 0.95rem;
            font-weight: 600;
            color: #e8eaed;
            margin-bottom: 8px;
            line-height: 1.3;
        }

        .scenario-subtitle {
            font-size: 0.75rem;
            color: #6b7280;
            margin-bottom: 10px;
        }

        .scenario-brief {
            font-size: 0.8rem;
            color: #9ca3af;
            line-height: 1.5;
        }

        .change-scenario-btn {
            width: 100%;
            padding: 10px;
            background: #0f1419;
            border: 1px solid #374151;
            border-radius: 6px;
            color: #00d4ff;
            font-size: 0.85rem;
            cursor: pointer;
            transition: all 0.2s;
        }

        .change-scenario-btn:hover {
            border-color: #00d4ff;
            background: rgba(0, 212, 255, 0.1);
        }

        .country-selector {
            background: #0f1419;
            border: 1px solid #374151;
            border-radius: 8px;
            padding: 16px;
            margin-bottom: 16px;
            transition: all 0.2s;
            cursor: pointer;
        }

        .country-selector:hover {
            border-color: #00d4ff;
        }

        .country-label {
            font-size: 0.75rem;
            color: #9ca3af;
            text-transform: uppercase;
            letter-spacing: 0.05em;
            margin-bottom: 10px;
        }

        .country-display {
            display: flex;
            align-items: center;
            gap: 12px;
            margin-bottom: 12px;
        }

        .country-flag {
            font-size: 2rem;
        }

        .country-name {
            font-size: 1.1rem;
            font-weight: 600;
            color: #e8eaed;
        }

        .role-selector {
            display: flex;
            align-items: center;
            gap: 10px;
        }

        .role-label {
            font-size: 0.75rem;
            color: #9ca3af;
        }

        .role-badge {
            padding: 6px 12px;
            background: rgba(0, 212, 255, 0.15);
            border: 1px solid #00d4ff;
            border-radius: 6px;
            color: #00d4ff;
            font-size: 0.8rem;
            font-weight: 600;
        }

        .player-indicator {
            background: linear-gradient(135deg, rgba(16, 185, 129, 0.15) 0%, rgba(5, 150, 105, 0.1) 100%);
            border: 2px solid #10b981;
            border-radius: 8px;
            padding: 14px;
            margin-top: 16px;
            display: flex;
            align-items: center;
            gap: 10px;
        }

        .player-icon {
            font-size: 1.5rem;
        }

        .player-text {
            font-size: 0.85rem;
            color: #10b981;
            font-weight: 600;
        }

        /* CENTER PANEL - SIMULATION FLOW */
        .simulation-display {
            background: #0f1419;
            overflow-y: auto;
            padding: 30px;
        }

        .round-display {
            background: linear-gradient(135deg, rgba(0, 212, 255, 0.1) 0%, rgba(0, 123, 255, 0.05) 100%);
            border: 2px solid #00d4ff;
            border-radius: 12px;
            padding: 28px;
            text-align: center;
            margin-bottom: 24px;
        }

        .round-label-top {
            font-size: 0.85rem;
            color: #9ca3af;
            text-transform: uppercase;
            letter-spacing: 0.1em;
            margin-bottom: 12px;
        }

        .round-number {
            font-size: 4.5rem;
            font-weight: 700;
            color: #00d4ff;
            line-height: 1;
            margin-bottom: 8px;
        }

        .round-label-bottom {
            font-size: 0.85rem;
            color: #9ca3af;
            text-transform: uppercase;
            letter-spacing: 0.1em;
        }

        .control-panel {
            background: #161b24;
            border: 1px solid #2d3748;
            border-radius: 12px;
            padding: 24px;
            margin-bottom: 24px;
        }

        .slider-control {
            margin-bottom: 20px;
        }

        .slider-label {
            display: flex;
            justify-content: space-between;
            align-items: center;
            margin-bottom: 10px;
            font-size: 0.85rem;
            color: #e8eaed;
        }

        .slider-value {
            background: rgba(0, 212, 255, 0.15);
            padding: 4px 10px;
            border-radius: 4px;
            color: #00d4ff;
            font-weight: 600;
        }

        .slider-input {
            width: 100%;
            height: 8px;
            border-radius: 4px;
            background: #0f1419;
            outline: none;
            -webkit-appearance: none;
        }

        .slider-input::-webkit-slider-thumb {
            -webkit-appearance: none;
            width: 20px;
            height: 20px;
            border-radius: 50%;
            background: #00d4ff;
            cursor: pointer;
            box-shadow: 0 0 10px rgba(0, 212, 255, 0.5);
        }

        .slider-input::-moz-range-thumb {
            width: 20px;
            height: 20px;
            border-radius: 50%;
            background: #00d4ff;
            cursor: pointer;
            border: none;
            box-shadow: 0 0 10px rgba(0, 212, 255, 0.5);
        }

        .toggle-control {
            display: flex;
            align-items: center;
            justify-content: space-between;
            padding: 14px;
            background: #0f1419;
            border: 1px solid #374151;
            border-radius: 8px;
            cursor: pointer;
            transition: all 0.2s;
        }

        .toggle-control:hover {
            border-color: #00d4ff;
            background: rgba(0, 212, 255, 0.05);
        }

        .toggle-label {
            font-size: 0.85rem;
            color: #e8eaed;
        }

        .toggle-switch {
            position: relative;
            width: 44px;
            height: 24px;
            background: #374151;
            border-radius: 12px;
            transition: all 0.3s;
        }

        .toggle-switch.active {
            background: #10b981;
        }

        .toggle-switch::after {
            content: '';
            position: absolute;
            top: 3px;
            left: 3px;
            width: 18px;
            height: 18px;
            background: white;
            border-radius: 50%;
            transition: all 0.3s;
        }

        .toggle-switch.active::after {
            left: 23px;
        }

        .nav-buttons {
            display: flex;
            gap: 12px;
        }

        .nav-btn {
            flex: 1;
            padding: 14px 18px;
            border-radius: 8px;
            cursor: pointer;
            font-size: 0.9rem;
            font-weight: 600;
            transition: all 0.2s;
            display: flex;
            align-items: center;
            justify-content: center;
            gap: 8px;
        }

        .nav-btn.primary {
            background: linear-gradient(135deg, #00d4ff 0%, #0080ff 100%);
            border: 2px solid #00d4ff;
            color: white;
        }

        .nav-btn.primary:hover {
            box-shadow: 0 0 20px rgba(0, 212, 255, 0.4);
            transform: translateY(-2px);
        }

        .nav-btn.danger {
            background: #0f1419;
            border: 2px solid #ef4444;
            color: #ef4444;
        }

        .nav-btn.danger:hover {
            background: rgba(239, 68, 68, 0.1);
            transform: translateY(-2px);
        }

        .outcomes-panel {
            background: linear-gradient(135deg, rgba(0, 212, 255, 0.03) 0%, rgba(0, 123, 255, 0.01) 100%);
            border: 1px solid rgba(0, 212, 255, 0.2);
            border-radius: 12px;
            padding: 24px;
            margin-bottom: 24px;
        }

        .outcomes-header {
            font-size: 0.85rem;
            text-transform: uppercase;
            letter-spacing: 0.1em;
            color: #00d4ff;
            margin-bottom: 20px;
            font-weight: 700;
            display: flex;
            align-items: center;
            gap: 8px;
        }

        .metrics-grid {
            display: grid;
            grid-template-columns: repeat(3, 1fr);
            gap: 16px;
        }

        .metric-card {
            background: #161b24;
            border: 1px solid #2d3748;
            border-radius: 10px;
            padding: 18px;
            position: relative;
            overflow: visible;
            transition: all 0.3s;
        }

        .metric-card:hover {
            border-color: #00d4ff;
            transform: translateY(-2px);
        }

        .metric-card::before {
            content: '';
            position: absolute;
            top: 0;
            left: 0;
            right: 0;
            height: 3px;
            background: linear-gradient(90deg, #00d4ff, #0080ff);
        }

        .metric-icon {
            font-size: 1.8rem;
            margin-bottom: 10px;
        }

        .metric-value {
            font-size: 2.2rem;
            font-weight: 700;
            color: #e8eaed;
            line-height: 1;
            margin-bottom: 6px;
        }

        .metric-value.reward {
            color: #10b981;
        }

        .metric-value.risk {
            color: #ef4444;
        }

        .metric-label {
            font-size: 0.8rem;
            color: #9ca3af;
            text-transform: uppercase;
            letter-spacing: 0.05em;
        }

        .metric-progress {
            margin-top: 10px;
            height: 6px;
            background: #0f1419;
            border-radius: 3px;
            overflow: visible;
        }

        .metric-progress-bar {
            height: 100%;
            background: linear-gradient(90deg, #00d4ff, #0080ff);
            border-radius: 3px;
            transition: width 0.5s ease;
        }

        .shock-alert {
            background: linear-gradient(135deg, rgba(239, 68, 68, 0.15) 0%, rgba(220, 38, 38, 0.1) 100%);
            border: 2px solid #ef4444;
            border-radius: 12px;
            padding: 20px;
            margin-bottom: 24px;
            animation: pulse-alert 2s infinite;
        }

        @keyframes pulse-alert {
            0%, 100% { box-shadow: 0 0 20px rgba(239, 68, 68, 0.3); }
            50% { box-shadow: 0 0 30px rgba(239, 68, 68, 0.5); }
        }

        .shock-header {
            display: flex;
            align-items: center;
            gap: 12px;
            font-size: 1.1rem;
            font-weight: 700;
            color: #ef4444;
            margin-bottom: 12px;
        }

        .shock-content {
            font-size: 0.9rem;
            color: #e8eaed;
            line-height: 1.6;
        }

        .adjudicator-panel {
            background: linear-gradient(135deg, rgba(0, 212, 255, 0.05) 0%, rgba(0, 123, 255, 0.02) 100%);
            border: 1px solid rgba(0, 212, 255, 0.2);
            border-radius: 12px;
            padding: 24px;
            margin-bottom: 24px;
        }

        .adjudicator-header {
            display: flex;
            align-items: center;
            gap: 12px;
            margin-bottom: 20px;
            font-size: 1.05rem;
            font-weight: 600;
            color: #00d4ff;
        }

        .adjudicator-metrics {
            display: grid;
            grid-template-columns: repeat(4, 1fr);
            gap: 16px;
        }

        .adj-metric {
            text-align: center;
        }

        .adj-metric-icon {
            font-size: 1.5rem;
            margin-bottom: 8px;
        }

        .adj-metric-value {
            font-size: 1.4rem;
            font-weight: 700;
            color: #e8eaed;
            margin-bottom: 4px;
        }

        .adj-metric-label {
            font-size: 0.75rem;
            color: #9ca3af;
            text-transform: uppercase;
            letter-spacing: 0.05em;
        }

        .deception-panel {
            background: linear-gradient(135deg, rgba(239, 68, 68, 0.05) 0%, rgba(185, 28, 28, 0.02) 100%);
            border: 1px solid rgba(239, 68, 68, 0.2);
            border-radius: 12px;
            padding: 24px;
            margin-bottom: 24px;
        }

        .deception-header {
            display: flex;
            align-items: center;
            gap: 12px;
            margin-bottom: 20px;
            font-size: 1.05rem;
            font-weight: 600;
            color: #ef4444;
        }

        .deception-list {
            display: flex;
            flex-direction: column;
            gap: 12px;
        }

        .deception-item {
            display: flex;
            justify-content: space-between;
            align-items: center;
            padding: 14px;
            background: #161b24;
            border: 1px solid #2d3748;
            border-radius: 8px;
            transition: all 0.2s;
        }

        .deception-item:hover {
            border-color: #ef4444;
        }

        .deception-actor {
            font-size: 0.9rem;
            color: #e8eaed;
            font-weight: 600;
        }

        .deception-risk {
            display: flex;
            align-items: center;
            gap: 10px;
        }

        .risk-bar {
            width: 100px;
            height: 8px;
            background: #0f1419;
            border-radius: 4px;
            overflow: visible;
        }

        .risk-bar-fill {
            height: 100%;
            background: linear-gradient(90deg, #fbbf24, #ef4444);
            border-radius: 4px;
            transition: width 0.5s ease;
        }

        .risk-percentage {
            font-size: 0.85rem;
            color: #9ca3af;
            min-width: 45px;
            text-align: right;
        }

        .status-indicator {
            padding: 4px 10px;
            border-radius: 4px;
            font-size: 0.75rem;
            font-weight: 600;
        }

        .status-indicator.safe {
            background: rgba(16, 185, 129, 0.15);
            color: #10b981;
        }

        .status-indicator.suspicious {
            background: rgba(239, 68, 68, 0.15);
            color: #ef4444;
        }

        .analysis-panel {
            background: #161b24;
            border-left: 1px solid #2d3748;
            overflow-y: auto;
            box-shadow: -2px 0 10px rgba(0,0,0,0.3);
            padding: 24px;
        }

        .analysis-section {
            margin-bottom: 28px;
            padding-bottom: 28px;
            border-bottom: 1px solid #2d3748;
        }

        .analysis-section:last-child {
            border-bottom: none;
        }

        .analysis-header {
            font-size: 1rem;
            font-weight: 600;
            color: #e8eaed;
            margin-bottom: 16px;
            display: flex;
            align-items: center;
            gap: 10px;
        }

        .comparison-table {
            width: 100%;
            border-collapse: collapse;
            margin-bottom: 20px;
        }

        .comparison-table th {
            background: #0f1419;
            padding: 12px;
            text-align: left;
            font-size: 0.8rem;
            color: #9ca3af;
            text-transform: uppercase;
            letter-spacing: 0.05em;
            border-bottom: 2px solid #2d3748;
        }

        .comparison-table td {
            padding: 12px;
            font-size: 0.85rem;
            color: #e8eaed;
            border-bottom: 1px solid #2d3748;
        }

        .comparison-table tr:hover td {
            background: rgba(0, 212, 255, 0.05);
        }

        .timeline-item {
            position: relative;
            padding-left: 24px;
            margin-bottom: 16px;
        }

        .timeline-item::before {
            content: '';
            position: absolute;
            left: 0;
            top: 6px;
            width: 12px;
            height: 12px;
            border-radius: 50%;
            background: #00d4ff;
            box-shadow: 0 0 8px rgba(0, 212, 255, 0.5);
        }

        .timeline-item::after {
            content: '';
            position: absolute;
            left: 5.5px;
            top: 18px;
            width: 1px;
            height: calc(100% + 4px);
            background: #2d3748;
        }

        .timeline-item:last-child::after {
            display: none;
        }

        .timeline-round {
            font-size: 0.75rem;
            color: #9ca3af;
            margin-bottom: 4px;
        }

        .timeline-content {
            font-size: 0.85rem;
            color: #e8eaed;
            line-height: 1.5;
        }

        .stats-grid {
            display: grid;
            grid-template-columns: 1fr 1fr;
            gap: 12px;
        }

        .stat-item {
            background: #0f1419;
            border: 1px solid #2d3748;
            border-radius: 8px;
            padding: 12px;
        }

        .stat-label {
            font-size: 0.75rem;
            color: #9ca3af;
            margin-bottom: 6px;
        }

        .stat-value {
            font-size: 1.3rem;
            font-weight: 700;
            color: #00d4ff;
        }

        .modal {
            display: none;
            position: fixed;
            top: 0;
            left: 0;
            width: 100%;
            height: 100%;
            background: rgba(0, 0, 0, 0.85);
            z-index: 1000;
            align-items: center;
            justify-content: center;
        }

        .modal.active {
            display: flex;
        }

        .modal-content {
            background: #161b24;
            border: 2px solid #2d3748;
            border-radius: 16px;
            padding: 32px;
            max-width: 800px;
            max-height: 80vh;
            overflow-y: auto;
            position: relative;
        }

        .close-modal {
            position: absolute;
            top: 16px;
            right: 16px;
            background: none;
            border: none;
            color: #9ca3af;
            font-size: 2rem;
            cursor: pointer;
            width: 40px;
            height: 40px;
            display: flex;
            align-items: center;
            justify-content: center;
            transition: all 0.2s;
        }

        .close-modal:hover {
            color: #ef4444;
            transform: rotate(90deg);
        }

        .modal-header {
            margin-bottom: 24px;
        }

        .modal-title {
            font-size: 1.5rem;
            color: #00d4ff;
            margin-bottom: 8px;
        }

        .modal-subtitle {
            font-size: 0.9rem;
            color: #9ca3af;
        }

        .scenario-grid {
            display: grid;
            gap: 16px;
        }

        .scenario-option {
            background: #0f1419;
            border: 2px solid #2d3748;
            border-radius: 12px;
            padding: 20px;
            cursor: pointer;
            transition: all 0.3s;
        }

        .scenario-option:hover,
        .scenario-option.selected {
            border-color: #00d4ff;
            background: rgba(0, 212, 255, 0.05);
        }

        .scenario-option-title {
            font-size: 1rem;
            font-weight: 600;
            color: #e8eaed;
            margin-bottom: 8px;
        }

        .scenario-option-desc {
            font-size: 0.85rem;
            color: #9ca3af;
            line-height: 1.5;
        }

        .country-grid {
            display: grid;
            grid-template-columns: repeat(3, 1fr);
            gap: 12px;
        }

        .country-option {
            background: #0f1419;
            border: 2px solid #2d3748;
            border-radius: 8px;
            padding: 16px;
            cursor: pointer;
            text-align: center;
            transition: all 0.2s;
        }

        .country-option:hover,
        .country-option.selected {
            border-color: #00d4ff;
            background: rgba(0, 212, 255, 0.05);
        }

        .country-option-flag {
            font-size: 2.5rem;
            margin-bottom: 8px;
        }

        .country-option-name {
            font-size: 0.85rem;
            color: #e8eaed;
            font-weight: 600;
        }

        .role-grid {
            display: grid;
            grid-template-columns: repeat(3, 1fr);
            gap: 10px;
            margin-top: 12px;
        }

        .role-option {
            background: #0f1419;
            border: 1px solid #2d3748;
            border-radius: 6px;
            padding: 10px;
            text-align: center;
            cursor: pointer;
            font-size: 0.8rem;
            color: #9ca3af;
            transition: all 0.2s;
        }

        .role-option:hover,
        .role-option.selected {
            border-color: #00d4ff;
            background: rgba(0, 212, 255, 0.1);
            color: #00d4ff;
        }

        ::-webkit-scrollbar {
            width: 10px;
        }

        ::-webkit-scrollbar-track {
            background: #0f1419;
        }

        ::-webkit-scrollbar-thumb {
            background: #2d3748;
            border-radius: 5px;
        }

        ::-webkit-scrollbar-thumb:hover {
            background: #374151;
        }
    </style>
</head>
<body>
    <div class="simulation-container">
        <div class="header-bar">
            <div class="platform-title">
                <div class="platform-icon">🎮</div>
                <div class="title-text">
                    <h1>Auracelle Charlie 3</h1>
                    <p>War Gaming Stress-Testing Simulation</p>
                </div>
            </div>
            <div class="header-actions">
                <div class="scenario-badge" onclick="openScenarioModal()">
                    <span id="currentScenario">EU AI Act</span>
                </div>
                <div class="status-badge">
                    <div class="status-dot"></div>
                    <span>Simulation Active</span>
                </div>
            </div>
        </div>

        <div class="dashboard-grid">
            <!-- Left Panel - Scenario Configuration -->
            <div class="config-panel">
                <div class="config-section">
                    <div class="section-header">
                        <span>📋</span>
                        <span>Policy Scenario</span>
                    </div>
                    <div class="scenario-card">
                        <div class="scenario-title">EU Artificial Intelligence Act (AI Act)</div>
                        <div class="scenario-subtitle">Regulation (EU) 2024/1689</div>
                        <div class="scenario-brief">
                            Risk-tiering, high-risk controls, governance obligations—perfect for stress-testing compliance, innovation tradeoffs, and cross-border adoption.
                        </div>
                    </div>
                    <button class="change-scenario-btn" onclick="openScenarioModal()">
                        Change Scenario
                    </button>
                </div>

                <div class="config-section">
                    <div class="section-header">
                        <span>🌍</span>
                        <span>Country A</span>
                    </div>
                    <div class="country-selector" onclick="openCountryModal('A')">
                        <div class="country-label">Selected Country</div>
                        <div class="country-display">
                            <div class="country-flag" id="countryAFlag">🇦🇪</div>
                            <div class="country-name" id="countryAName">Dubai</div>
                        </div>
                        <div class="role-selector">
                            <div class="role-label">Role:</div>
                            <div class="role-badge" id="countryARole">Governance</div>
                        </div>
                    </div>
                </div>

                <div class="config-section">
                    <div class="section-header">
                        <span>🌍</span>
                        <span>Country B</span>
                    </div>
                    <div class="country-selector" onclick="openCountryModal('B')">
                        <div class="country-label">Selected Country</div>
                        <div class="country-display">
                            <div class="country-flag" id="countryBFlag">🇬🇧</div>
                            <div class="country-name" id="countryBName">United Kingdom</div>
                        </div>
                        <div class="role-selector">
                            <div class="role-label">Role:</div>
                            <div class="role-badge" id="countryBRole">Governance</div>
                        </div>
                    </div>
                </div>

                <div class="config-section">
                    <div class="section-header">
                        <span>🎖️</span>
                        <span>Your Role</span>
                    </div>
                    <div class="player-indicator">
                        <div class="player-icon">👤</div>
                        <div>
                            <div style="font-size: 0.75rem; color: #9ca3af; margin-bottom: 4px;">You represent:</div>
                            <div class="player-text" id="playerCountry">Dubai</div>
                        </div>
                    </div>
                </div>
            </div>

            <!-- Center Panel - SIMULATION FLOW -->
            <div class="simulation-display">
                <!-- 1. SIMULATION CONTROL -->
                <div class="round-display">
                    <div class="round-label-top">Current Round</div>
                    <div class="round-number" id="roundNumber">1</div>
                    <div class="round-label-bottom">Round</div>
                </div>

                <div class="control-panel">
                    <div class="section-header">
                        <span>⚙️</span>
                        <span>Episode Settings</span>
                    </div>

                    <div class="slider-control">
                        <div class="slider-label">
                            <span>Episode Length</span>
                            <span class="slider-value" id="episodeLengthValue">5 rounds</span>
                        </div>
                        <input type="range" class="slider-input" id="episodeLength" min="1" max="30" value="5">
                    </div>

                    <div class="toggle-control" onclick="toggleStochastic()">
                        <span class="toggle-label">Stochastic Exploration</span>
                        <div class="toggle-switch" id="stochasticToggle"></div>
                    </div>
                </div>

                <div class="control-panel" style="border: 2px solid #00d4ff;">
                    <div class="section-header">
                        <span>🎮</span>
                        <span>Controls</span>
                    </div>
                    <div class="nav-buttons">
                        <button class="nav-btn primary" onclick="nextRound()">
                            <span>▶️</span>
                            <span>Next Round</span>
                        </button>
                        <button class="nav-btn danger" onclick="resetEpisode()">
                            <span>🔄</span>
                            <span>Reset</span>
                        </button>
                    </div>
                </div>

                <!-- 2. SIMULATION OUTCOMES -->
                <div class="outcomes-panel">
                    <div class="outcomes-header">
                        <span>📊</span>
                        <span>Round Outcomes</span>
                    </div>

                    <div class="metrics-grid">
                        <div class="metric-card">
                            <div class="metric-icon">🏆</div>
                            <div class="metric-value reward" id="rewardValue">29.0</div>
                            <div class="metric-label">Reward</div>
                            <div class="metric-progress">
                                <div class="metric-progress-bar" id="rewardBar" style="width: 29%"></div>
                            </div>
                        </div>

                        <div class="metric-card">
                            <div class="metric-icon">⚠️</div>
                            <div class="metric-value risk" id="riskValue">50.0</div>
                            <div class="metric-label">Risk</div>
                            <div class="metric-progress">
                                <div class="metric-progress-bar" id="riskBar" style="width: 50%; background: linear-gradient(90deg, #fbbf24, #ef4444);"></div>
                            </div>
                        </div>

                        <div class="metric-card">
                            <div class="metric-icon">📏</div>
                            <div class="metric-value" id="episodeProgress">1/5</div>
                            <div class="metric-label">Episode Progress</div>
                            <div class="metric-progress">
                                <div class="metric-progress-bar" id="episodeBar" style="width: 20%"></div>
                            </div>
                        </div>
                    </div>
                </div>

                <!-- 3. SHOCK EVENTS -->
                <div class="shock-alert" id="shockAlert" style="display: none;">
                    <div class="shock-header">
                        <span>💥</span>
                        <span id="shockTitle">EXTERNAL SHOCK</span>
                    </div>
                    <div class="shock-content" id="shockContent">
                        Unexpected geopolitical development detected.
                    </div>
                </div>

                <!-- 4. AI ADJUDICATOR -->
                <div class="adjudicator-panel">
                    <div class="adjudicator-header">
                        <span>🤖</span>
                        <span>AI Agentic Adjudicator Status</span>
                    </div>
                    <div class="adjudicator-metrics">
                        <div class="adj-metric">
                            <div class="adj-metric-icon" id="tensionIcon">🟢</div>
                            <div class="adj-metric-value" id="tensionValue">38.5%</div>
                            <div class="adj-metric-label">Tension</div>
                        </div>
                        <div class="adj-metric">
                            <div class="adj-metric-icon">🎲</div>
                            <div class="adj-metric-value" id="confidenceValue">88.5%</div>
                            <div class="adj-metric-label">Confidence</div>
                        </div>
                        <div class="adj-metric">
                            <div class="adj-metric-icon">📊</div>
                            <div class="adj-metric-value" id="alignmentValue">0%</div>
                            <div class="adj-metric-label">Alignment</div>
                        </div>
                        <div class="adj-metric">
                            <div class="adj-metric-icon">✅</div>
                            <div class="adj-metric-value">Active</div>
                            <div class="adj-metric-label">Real Data</div>
                        </div>
                    </div>
                </div>

                <!-- 5. DECEPTION DETECTION -->
                <div class="deception-panel">
                    <div class="deception-header">
                        <span>🎭</span>
                        <span>Deception Detection</span>
                    </div>
                    <div class="deception-list">
                        <div class="deception-item">
                            <div class="deception-actor"><span id="deceptionCountryA">🇦🇪 Dubai</span></div>
                            <div class="deception-risk">
                                <div class="risk-bar">
                                    <div class="risk-bar-fill" id="deceptionBarA" style="width: 22%"></div>
                                </div>
                                <div class="risk-percentage" id="deceptionPercentA">22.0%</div>
                                <div class="status-indicator safe" id="deceptionStatusA">✅ Consistent</div>
                            </div>
                        </div>
                        <div class="deception-item">
                            <div class="deception-actor"><span id="deceptionCountryB">🇬🇧 United Kingdom</span></div>
                            <div class="deception-risk">
                                <div class="risk-bar">
                                    <div class="risk-bar-fill" id="deceptionBarB" style="width: 35%"></div>
                                </div>
                                <div class="risk-percentage" id="deceptionPercentB">35.0%</div>
                                <div class="status-indicator safe" id="deceptionStatusB">✅ Consistent</div>
                            </div>
                        </div>
                    </div>
                </div>
            </div>

            <!-- Right Panel - Round History & Analysis -->
            <div class="analysis-panel">
                <div class="analysis-section">
                    <div class="analysis-header">
                        <span>🆚</span>
                        <span>Country Comparison</span>
                    </div>
                    <table class="comparison-table">
                        <thead>
                            <tr>
                                <th>Metric</th>
                                <th id="comparisonHeaderA">Dubai</th>
                                <th id="comparisonHeaderB">UK</th>
                            </tr>
                        </thead>
                        <tbody>
                            <tr>
                                <td>GDP (Trillion USD)</td>
                                <td id="gdpA">$0.51T</td>
                                <td id="gdpB">$3.34T</td>
                            </tr>
                            <tr>
                                <td>Military Exp (% GDP)</td>
                                <td id="milA">5.7%</td>
                                <td id="milB">2.3%</td>
                            </tr>
                            <tr>
                                <td>Internet Penetration</td>
                                <td id="internetA">99.0%</td>
                                <td id="internetB">96.6%</td>
                            </tr>
                            <tr>
                                <td>Influence Score</td>
                                <td id="influenceA">7.2</td>
                                <td id="influenceB">8.9</td>
                            </tr>
                            <tr>
                                <td>Policy Position</td>
                                <td id="positionA">Moderate</td>
                                <td id="positionB">Restrictive</td>
                            </tr>
                        </tbody>
                    </table>
                </div>

                <div class="analysis-section">
                    <div class="analysis-header">
                        <span>📈</span>
                        <span>Round History</span>
                    </div>
                    <div id="roundTimeline">
                        <div class="timeline-item">
                            <div class="timeline-round">Round 1</div>
                            <div class="timeline-content">
                                Initial policy positions established. Moderate tension between Dubai's governance approach and UK's regulatory framework.
                            </div>
                        </div>
                    </div>
                </div>

                <div class="analysis-section">
                    <div class="analysis-header">
                        <span>🎯</span>
                        <span>Strategic Assessment</span>
                    </div>
                    <div style="font-size: 0.85rem; color: #9ca3af; line-height: 1.6;">
                        <p style="margin-bottom: 12px;">
                            <strong style="color: #00d4ff;">Current Scenario:</strong>
                            EU AI Act implementation requires balancing innovation incentives with comprehensive risk management frameworks.
                        </p>
                        <p style="margin-bottom: 12px;">
                            <strong style="color: #00d4ff;">Position Analysis:</strong>
                            Dubai's governance-focused approach contrasts with UK's emphasis on regulatory alignment. Potential for collaborative standard-setting.
                        </p>
                        <p>
                            <strong style="color: #00d4ff;">Recommendation:</strong>
                            Explore tiered compliance mechanisms that accommodate different implementation capacities while maintaining core safety standards.
                        </p>
                    </div>
                </div>

                <div class="analysis-section">
                    <div class="analysis-header">
                        <span>📊</span>
                        <span>Session Statistics</span>
                    </div>
                    <div class="stats-grid">
                        <div class="stat-item">
                            <div class="stat-label">Avg Reward</div>
                            <div class="stat-value" id="avgReward">29.0</div>
                        </div>
                        <div class="stat-item">
                            <div class="stat-label">Avg Risk</div>
                            <div class="stat-value" id="avgRisk">50.0</div>
                        </div>
                        <div class="stat-item">
                            <div class="stat-label">Rounds Played</div>
                            <div class="stat-value" id="roundsPlayed">1</div>
                        </div>
                        <div class="stat-item">
                            <div class="stat-label">Shocks</div>
                            <div class="stat-value" id="shocksTriggered">0</div>
                        </div>
                    </div>
                </div>
            </div>
        </div>
    </div>

    <!-- Scenario Selection Modal -->
    <div class="modal" id="scenarioModal">
        <div class="modal-content">
            <button class="close-modal" onclick="closeScenarioModal()">×</button>
            <div class="modal-header">
                <h2 class="modal-title">Select Policy Scenario</h2>
                <p class="modal-subtitle">Choose the governance framework to stress-test</p>
            </div>
            <div class="scenario-grid">
                <div class="scenario-option selected" onclick="selectScenario(this, 'EU AI Act', 'EU Artificial Intelligence Act (AI Act)', 'Regulation (EU) 2024/1689', 'Risk-tiering, high-risk controls, governance obligations—perfect for stress-testing compliance, innovation tradeoffs, and cross-border adoption.')">
                    <div class="scenario-option-title">EU Artificial Intelligence Act (AI Act)</div>
                    <div class="scenario-option-desc">Regulation (EU) 2024/1689 • Risk-tiering, high-risk controls, governance obligations</div>
                </div>
                <div class="scenario-option" onclick="selectScenario(this, 'US Executive Order', 'US Executive Order on Safe AI', 'EO 14110 (Oct 2023)', 'Federal AI governance, safety standards, and international coordination frameworks.')">
                    <div class="scenario-option-title">US Executive Order on Safe AI</div>
                    <div class="scenario-option-desc">EO 14110 (Oct 2023) • Federal standards, safety testing, international cooperation</div>
                </div>
                <div class="scenario-option" onclick="selectScenario(this, 'UK AI Regulation', 'UK Pro-Innovation AI Regulation', 'White Paper Framework', 'Principles-based approach emphasizing sector-specific regulation and innovation enablement.')">
                    <div class="scenario-option-title">UK Pro-Innovation AI Regulation</div>
                    <div class="scenario-option-desc">White Paper Framework • Principles-based, sector-specific approach</div>
                </div>
                <div class="scenario-option" onclick="selectScenario(this, 'China AI Governance', 'China Generative AI Measures', 'CAC Regulations 2023', 'State oversight, content controls, algorithm registration, and national security focus.')">
                    <div class="scenario-option-title">China Generative AI Measures</div>
                    <div class="scenario-option-desc">CAC Regulations 2023 • Algorithm governance, content regulation</div>
                </div>
                <div class="scenario-option" onclick="selectScenario(this, 'OECD AI Principles', 'OECD AI Principles', 'May 2019 Framework', 'International consensus on responsible AI stewardship and human-centric values.')">
                    <div class="scenario-option-title">OECD AI Principles</div>
                    <div class="scenario-option-desc">May 2019 Framework • International consensus, values-based approach</div>
                </div>
                <div class="scenario-option" onclick="selectScenario(this, 'Bletchley Declaration', 'Bletchley Declaration', 'UK AI Safety Summit 2023', 'International cooperation on frontier AI risks and safety research coordination.')">
                    <div class="scenario-option-title">Bletchley Declaration</div>
                    <div class="scenario-option-desc">UK AI Safety Summit 2023 • Frontier AI risks, international cooperation</div>
                </div>
            </div>
        </div>
    </div>

    <!-- Country Selection Modal -->
    <div class="modal" id="countryModal">
        <div class="modal-content">
            <button class="close-modal" onclick="closeCountryModal()">×</button>
            <div class="modal-header">
                <h2 class="modal-title">Select Country <span id="countrySlot"></span></h2>
                <p class="modal-subtitle">Choose actor and assign strategic role</p>
            </div>

            <div style="margin-bottom: 24px;">
                <div style="font-size: 0.85rem; color: #9ca3af; margin-bottom: 12px;">Select Country:</div>
                <div class="country-grid" id="countryGrid">
                    <!-- Countries populated by JavaScript -->
                </div>
            </div>

            <div>
                <div style="font-size: 0.85rem; color: #9ca3af; margin-bottom: 12px;">Assign Role:</div>
                <div class="role-grid" id="roleGrid">
                    <div class="role-option" onclick="selectRole(this, 'Governance')">Governance</div>
                    <div class="role-option" onclick="selectRole(this, 'MilitaryAI')">Military AI</div>
                    <div class="role-option" onclick="selectRole(this, 'DataPrivacy')">Data Privacy</div>
                    <div class="role-option" onclick="selectRole(this, 'ExportControl')">Export Control</div>
                    <div class="role-option" onclick="selectRole(this, 'Diplomacy')">Diplomacy</div>
                    <div class="role-option" onclick="selectRole(this, 'StandardSetting')">Standard Setting</div>
                    <div class="role-option" onclick="selectRole(this, 'Surveillance')">Surveillance</div>
                    <div class="role-option" onclick="selectRole(this, 'Trade')">Trade</div>
                    <div class="role-option" onclick="selectRole(this, 'TechAlliance')">Tech Alliance</div>
                </div>
            </div>

            <div style="margin-top: 24px; display: flex; gap: 12px;">
                <button class="nav-btn" onclick="closeCountryModal()" style="flex: 1; border: 2px solid #374151; background: #0f1419; color: #e8eaed;">Cancel</button>
                <button class="nav-btn primary" onclick="confirmCountrySelection()" style="flex: 1;">Confirm Selection</button>
            </div>
        </div>
    </div>

    <script>
        // ===== STATE MANAGEMENT =====
        let state = {
            round: 1,
            episodeLength: 5,
            stochastic: false,
            reward: 29.0,
            risk: 50.0,
            tension: 38.5,
            confidence: 88.5,
            alignment: 0,
            history: [],
            shocksTriggered: 0,
            scenario: {
                short: 'EU AI Act',
                title: 'EU Artificial Intelligence Act (AI Act)',
                subtitle: 'Regulation (EU) 2024/1689',
                brief: 'Risk-tiering, high-risk controls, governance obligations—perfect for stress-testing compliance, innovation tradeoffs, and cross-border adoption.'
            },
            countryA: {
                name: 'Dubai',
                flag: '🇦🇪',
                role: 'Governance',
                gdp: 0.51,
                mil: 5.7,
                internet: 99.0,
                influence: 7.2,
                position: 'Moderate'
            },
            countryB: {
                name: 'United Kingdom',
                flag: '🇬🇧',
                role: 'Governance',
                gdp: 3.34,
                mil: 2.3,
                internet: 96.6,
                influence: 8.9,
                position: 'Restrictive'
            },
            player: 'A',
            currentModalSlot: null,
            tempCountry: null,
            tempRole: 'Governance'
        };

        const countries = [
            { name: 'United States', flag: '🇺🇸', gdp: 27.36, mil: 3.4, internet: 91.8, influence: 9.8, position: 'Moderate' },
            { name: 'China', flag: '🇨🇳', gdp: 17.89, mil: 1.7, internet: 73.7, influence: 9.5, position: 'Restrictive' },
            { name: 'European Union', flag: '🇪🇺', gdp: 16.64, mil: 1.5, internet: 89.7, influence: 9.2, position: 'Restrictive' },
            { name: 'United Kingdom', flag: '🇬🇧', gdp: 3.34, mil: 2.3, internet: 96.6, influence: 8.9, position: 'Restrictive' },
            { name: 'India', flag: '🇮🇳', gdp: 3.73, mil: 2.4, internet: 54.4, influence: 8.5, position: 'Permissive' },
            { name: 'Japan', flag: '🇯🇵', gdp: 4.21, mil: 1.1, internet: 93.8, influence: 8.7, position: 'Moderate' },
            { name: 'Germany', flag: '🇩🇪', gdp: 4.31, mil: 1.5, internet: 91.0, influence: 8.8, position: 'Restrictive' },
            { name: 'France', flag: '🇫🇷', gdp: 3.05, mil: 1.9, internet: 85.5, influence: 8.6, position: 'Moderate' },
            { name: 'Canada', flag: '🇨🇦', gdp: 2.14, mil: 1.3, internet: 92.7, influence: 8.3, position: 'Moderate' },
            { name: 'South Korea', flag: '🇰🇷', gdp: 1.71, mil: 2.8, internet: 97.2, influence: 8.4, position: 'Moderate' },
            { name: 'Australia', flag: '🇦🇺', gdp: 1.69, mil: 2.0, internet: 89.6, influence: 8.1, position: 'Moderate' },
            { name: 'Brazil', flag: '🇧🇷', gdp: 2.17, mil: 1.1, internet: 81.3, influence: 7.8, position: 'Permissive' },
            { name: 'Israel', flag: '🇮🇱', gdp: 0.52, mil: 4.5, internet: 88.9, influence: 7.9, position: 'Moderate' },
            { name: 'Singapore', flag: '🇸🇬', gdp: 0.52, mil: 3.2, internet: 92.0, influence: 7.7, position: 'Permissive' },
            { name: 'Dubai', flag: '🇦🇪', gdp: 0.51, mil: 5.7, internet: 99.0, influence: 7.2, position: 'Moderate' },
            { name: 'Saudi Arabia', flag: '🇸🇦', gdp: 1.11, mil: 7.4, internet: 99.0, influence: 7.5, position: 'Restrictive' },
            { name: 'Russia', flag: '🇷🇺', gdp: 2.24, mil: 4.1, internet: 88.2, influence: 8.0, position: 'Restrictive' },
            { name: 'Turkey', flag: '🇹🇷', gdp: 1.03, mil: 1.3, internet: 82.6, influence: 7.3, position: 'Moderate' },
            { name: 'Mexico', flag: '🇲🇽', gdp: 1.81, mil: 0.5, internet: 75.5, influence: 7.4, position: 'Permissive' },
            { name: 'Indonesia', flag: '🇮🇩', gdp: 1.39, mil: 0.8, internet: 77.0, influence: 7.6, position: 'Permissive' }
        ];

        function init() {
            updateAllDisplays();
            document.getElementById('episodeLength').addEventListener('input', function(e) {
                state.episodeLength = parseInt(e.target.value);
                document.getElementById('episodeLengthValue').textContent = `${state.episodeLength} rounds`;
                updateEpisodeProgress();
            });
        }

        function openScenarioModal() {
            document.getElementById('scenarioModal').classList.add('active');
        }

        function closeScenarioModal() {
            document.getElementById('scenarioModal').classList.remove('active');
        }

        function selectScenario(element, short, title, subtitle, brief) {
            document.querySelectorAll('.scenario-option').forEach(opt => opt.classList.remove('selected'));
            element.classList.add('selected');

            state.scenario = { short, title, subtitle, brief };

            document.getElementById('currentScenario').textContent = short;
            document.querySelector('.scenario-title').textContent = title;
            document.querySelector('.scenario-subtitle').textContent = subtitle;
            document.querySelector('.scenario-brief').textContent = brief;

            setTimeout(() => closeScenarioModal(), 300);
        }

        function openCountryModal(slot) {
            state.currentModalSlot = slot;
            document.getElementById('countrySlot').textContent = slot;

            const grid = document.getElementById('countryGrid');
            grid.innerHTML = '';

            countries.forEach(country => {
                const div = document.createElement('div');
                div.className = 'country-option';
                if ((slot === 'A' && country.name === state.countryA.name) ||
                    (slot === 'B' && country.name === state.countryB.name)) {
                    div.classList.add('selected');
                    state.tempCountry = country;
                }
                div.innerHTML = `
                    <div class="country-option-flag">${country.flag}</div>
                    <div class="country-option-name">${country.name}</div>
                `;
                div.onclick = function() {
                    document.querySelectorAll('.country-option').forEach(opt => opt.classList.remove('selected'));
                    div.classList.add('selected');
                    state.tempCountry = country;
                };
                grid.appendChild(div);
            });

            const currentRole = slot === 'A' ? state.countryA.role : state.countryB.role;
            state.tempRole = currentRole;
            document.querySelectorAll('.role-option').forEach(opt => {
                opt.classList.remove('selected');
                if (opt.textContent === currentRole) {
                    opt.classList.add('selected');
                }
            });

            document.getElementById('countryModal').classList.add('active');
        }

        function closeCountryModal() {
            document.getElementById('countryModal').classList.remove('active');
        }

        function selectRole(element, role) {
            document.querySelectorAll('.role-option').forEach(opt => opt.classList.remove('selected'));
            element.classList.add('selected');
            state.tempRole = role;
        }

        function confirmCountrySelection() {
            if (!state.tempCountry) return;

            const slot = state.currentModalSlot;
            const country = {
                name: state.tempCountry.name,
                flag: state.tempCountry.flag,
                role: state.tempRole,
                gdp: state.tempCountry.gdp,
                mil: state.tempCountry.mil,
                internet: state.tempCountry.internet,
                influence: state.tempCountry.influence,
                position: state.tempCountry.position
            };

            if (slot === 'A') {
                state.countryA = country;
            } else {
                state.countryB = country;
            }

            updateAllDisplays();
            closeCountryModal();
        }

        function nextRound() {
            state.round++;

            if (state.stochastic) {
                state.reward = Math.max(0, Math.min(100, state.reward + (Math.random() - 0.5) * 20));
                state.risk = Math.max(0, Math.min(100, state.risk + (Math.random() - 0.5) * 15));
                state.tension = Math.max(0, Math.min(100, state.tension + (Math.random() - 0.5) * 10));
                state.confidence = Math.max(0, Math.min(100, state.confidence + (Math.random() - 0.5) * 8));
                state.alignment = Math.max(0, Math.min(100, state.alignment + (Math.random() - 0.5) * 12));
            } else {
                state.reward += (Math.random() - 0.3) * 8;
                state.risk += (Math.random() - 0.6) * 6;
                state.tension += (Math.random() - 0.5) * 5;
                state.confidence += (Math.random() - 0.4) * 4;
                state.alignment += Math.random() * 10;
            }

            state.reward = Math.max(0, Math.min(100, state.reward));
            state.risk = Math.max(0, Math.min(100, state.risk));
            state.tension = Math.max(0, Math.min(100, state.tension));
            state.confidence = Math.max(0, Math.min(100, state.confidence));
            state.alignment = Math.max(0, Math.min(100, state.alignment));

            state.history.push({
                round: state.round,
                reward: state.reward,
                risk: state.risk,
                tension: state.tension,
                confidence: state.confidence
            });

            if (Math.random() < 0.1) {
                triggerShockEvent();
            } else {
                hideShockEvent();
            }

            updateAllDisplays();
            addTimelineEntry();
        }

        function resetEpisode() {
            if (confirm('Reset the entire episode? All progress will be lost.')) {
                state.round = 1;
                state.reward = 29.0;
                state.risk = 50.0;
                state.tension = 38.5;
                state.confidence = 88.5;
                state.alignment = 0;
                state.history = [];
                state.shocksTriggered = 0;

                document.getElementById('roundTimeline').innerHTML = `
                    <div class="timeline-item">
                        <div class="timeline-round">Round 1</div>
                        <div class="timeline-content">
                            Initial policy positions established. Moderate tension between ${state.countryA.name}'s governance approach and ${state.countryB.name}'s regulatory framework.
                        </div>
                    </div>
                `;

                hideShockEvent();
                updateAllDisplays();
            }
        }

        function toggleStochastic() {
            state.stochastic = !state.stochastic;
            const toggle = document.getElementById('stochasticToggle');
            if (state.stochastic) {
                toggle.classList.add('active');
            } else {
                toggle.classList.remove('active');
            }
        }

        function triggerShockEvent() {
            state.shocksTriggered++;
            const shockEvents = [
                {
                    title: "💥 EXTERNAL SHOCK: Trade Embargo Announced",
                    content: "Major economy announces sudden export restrictions on critical AI components. Supply chain disruption imminent."
                },
                {
                    title: "💥 REAL-WORLD TRIGGERED SHOCK: Cyber Attack",
                    content: "Large-scale cyber incident targeting AI research infrastructure. International response coordination required."
                },
                {
                    title: "💥 EXTERNAL SHOCK: Regulatory Framework Shift",
                    content: "Unexpected regulatory announcement by coalition of nations. Policy harmonization window closing."
                },
                {
                    title: "💥 EXTERNAL SHOCK: Breakthrough Technology",
                    content: "Unexpected AI capability demonstration shifts strategic landscape. Competitive dynamics altered."
                }
            ];

            const shock = shockEvents[Math.floor(Math.random() * shockEvents.length)];
            document.getElementById('shockTitle').textContent = shock.title.replace('💥 ', '');
            document.getElementById('shockContent').textContent = shock.content;
            document.getElementById('shockAlert').style.display = 'block';

            state.risk += 15;
            state.tension += 10;
            state.confidence -= 8;
        }

        function hideShockEvent() {
            document.getElementById('shockAlert').style.display = 'none';
        }

        function updateAllDisplays() {
            document.getElementById('roundNumber').textContent = state.round;

            document.getElementById('rewardValue').textContent = state.reward.toFixed(1);
            document.getElementById('riskValue').textContent = state.risk.toFixed(1);
            document.getElementById('rewardBar').style.width = state.reward + '%';
            document.getElementById('riskBar').style.width = state.risk + '%';

            document.getElementById('tensionValue').textContent = state.tension.toFixed(1) + '%';
            document.getElementById('confidenceValue').textContent = state.confidence.toFixed(1) + '%';
            document.getElementById('alignmentValue').textContent = Math.floor(state.alignment) + '%';

            const tensionIcon = document.getElementById('tensionIcon');
            if (state.tension > 70) tensionIcon.textContent = '🔴';
            else if (state.tension > 40) tensionIcon.textContent = '🟡';
            else tensionIcon.textContent = '🟢';

            document.getElementById('countryAFlag').textContent = state.countryA.flag;
            document.getElementById('countryAName').textContent = state.countryA.name;
            document.getElementById('countryARole').textContent = state.countryA.role;

            document.getElementById('countryBFlag').textContent = state.countryB.flag;
            document.getElementById('countryBName').textContent = state.countryB.name;
            document.getElementById('countryBRole').textContent = state.countryB.role;

            document.getElementById('playerCountry').textContent = state.player === 'A' ? state.countryA.name : state.countryB.name;

            document.getElementById('comparisonHeaderA').textContent = state.countryA.name;
            document.getElementById('comparisonHeaderB').textContent = state.countryB.name;
            document.getElementById('gdpA').textContent = `$${state.countryA.gdp.toFixed(2)}T`;
            document.getElementById('gdpB').textContent = `$${state.countryB.gdp.toFixed(2)}T`;
            document.getElementById('milA').textContent = `${state.countryA.mil.toFixed(1)}%`;
            document.getElementById('milB').textContent = `${state.countryB.mil.toFixed(1)}%`;
            document.getElementById('internetA').textContent = `${state.countryA.internet.toFixed(1)}%`;
            document.getElementById('internetB').textContent = `${state.countryB.internet.toFixed(1)}%`;
            document.getElementById('influenceA').textContent = state.countryA.influence.toFixed(1);
            document.getElementById('influenceB').textContent = state.countryB.influence.toFixed(1);
            document.getElementById('positionA').textContent = state.countryA.position;
            document.getElementById('positionB').textContent = state.countryB.position;

            document.getElementById('deceptionCountryA').textContent = `${state.countryA.flag} ${state.countryA.name}`;
            document.getElementById('deceptionCountryB').textContent = `${state.countryB.flag} ${state.countryB.name}`;

            updateEpisodeProgress();
            updateHistoryStats();
        }

        function updateEpisodeProgress() {
            const progress = ((state.round - 1) % state.episodeLength) + 1;
            document.getElementById('episodeProgress').textContent = `${progress}/${state.episodeLength}`;
            document.getElementById('episodeBar').style.width = (progress / state.episodeLength * 100) + '%';
        }

        function updateHistoryStats() {
            if (state.history.length > 0) {
                const avgReward = state.history.reduce((sum, h) => sum + h.reward, 0) / state.history.length;
                const avgRisk = state.history.reduce((sum, h) => sum + h.risk, 0) / state.history.length;

                document.getElementById('avgReward').textContent = avgReward.toFixed(1);
                document.getElementById('avgRisk').textContent = avgRisk.toFixed(1);
            } else {
                document.getElementById('avgReward').textContent = state.reward.toFixed(1);
                document.getElementById('avgRisk').textContent = state.risk.toFixed(1);
            }

            document.getElementById('roundsPlayed').textContent = state.round;
            document.getElementById('shocksTriggered').textContent = state.shocksTriggered;
        }

        function addTimelineEntry() {
            const timeline = document.getElementById('roundTimeline');

            const narratives = [
                `Policy alignment improving between ${state.countryA.name} and ${state.countryB.name}. Cooperative signals strengthening.`,
                `Tension escalating in bilateral negotiations. Divergent strategic priorities detected.`,
                `${state.countryA.name}'s ${state.countryA.role} approach creating friction with ${state.countryB.name}'s position.`,
                `Breakthrough in stakeholder coordination. Risk mitigation strategies proving effective.`,
                `Strategic ambiguity detected. Deception indicators elevated for one or more actors.`,
                `Coalition dynamics shifting. Multi-lateral considerations becoming dominant factor.`,
                `Compliance frameworks showing strain. Recommend immediate policy adjustment protocols.`,
                `Alignment metrics improving significantly. Reward optimization trajectory enhanced.`
            ];

            const narrative = narratives[Math.floor(Math.random() * narratives.length)];

            const entry = document.createElement('div');
            entry.className = 'timeline-item';
            entry.innerHTML = `
                <div class="timeline-round">Round ${state.round}</div>
                <div class="timeline-content">${narrative}</div>
            `;

            timeline.appendChild(entry);
            timeline.scrollTop = timeline.scrollHeight;
        }

        window.onload = init;
    </script>
</body>
</html>
'''

with open('simulation_vertical.html', 'w', encoding='utf-8') as f:
    f.write(html_content)

print('✅ Simulation HTML file created (vertical layout, Round History first)')

✅ Simulation HTML file created (vertical layout, Round History first)


In [4]:
# ========================================
# CREATE 3D INFLUENCE MAP PAGE
# ========================================
influence_map_code = '''
import streamlit as st
import streamlit.components.v1 as components
import pandas as pd

st.set_page_config(page_title="3D Influence Map", page_icon="🧬", layout="wide")

# Password protection
if not st.session_state.get("authenticated", False):
    st.warning("Please log in first.")
    st.switch_page("app.py")

# =============================================================================
# PAGE HEADER
# =============================================================================

st.title("🧬 3D AlphaFold-Style Influence Map")
st.markdown("""
<div style='background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            padding: 20px; border-radius: 10px; margin-bottom: 20px;'>
    <h3 style='color: white; margin: 0;'>External Policy Pressures & Internal Cultural Forces</h3>
    <p style='color: white; margin: 5px 0 0 0; font-size: 14px;'>
        Interactive 3D Visualization of AI Governance Influences
    </p>
</div>
""", unsafe_allow_html=True)

st.markdown("""
### 🎯 What This Shows

Unlike traditional network visualizations that show static relationships, this **AlphaFold-style influence map**
reveals how **external policy pressures** (like GDPR, export controls) and **internal cultural forces**
(like democratic values, tech nationalism) shape countries' AI governance positions in 3D space.

**The 3D Space:**
- **X-axis**: Economic strength (GDP)
- **Y-axis**: Influence score (0-1)
- **Z-axis**: Policy position alignment

**Each node represents:**
- Countries (US, EU, China, UK, Japan, Dubai, etc.)
- Organizations (NATO)

**Visualization Elements:**
- 🔵 **Blue Arrows** = External policy pressures (GDPR, sanctions, export controls)
- 🟣 **Purple Spheres** = Internal cultural forces (democratic norms, tech nationalism)
- 📊 **Node Size** = Relative influence
- 🎨 **Node Color** = Alignment clusters

**Interactive Features:**
- Rotate: Click and drag
- Zoom: Scroll
- Animate: Watch policy convergence over time
""")

# =============================================================================
# DATA DEFINITIONS (same as original)
# =============================================================================

default_data = {
    "European Union": {"gdp": 15.0, "influence": 0.90, "position": "Strict data protection (GDPR)", "mil_exp": 1.5, "internet": 89.0, "cultural_alignment": "Western"},
    "Dubai": {"gdp": 0.5, "influence": 0.7, "position": "Moderate regulatory stance", "mil_exp": 5.6, "internet": 99.0, "cultural_alignment": "Western-Middle East hybrid"},
    "United Kingdom": {"gdp": 3.2, "influence": 0.85, "position": "Supports EU-style data protection", "mil_exp": 2.2, "internet": 96.0, "cultural_alignment": "Western"},
    "United States": {"gdp": 21.0, "influence": 0.95, "position": "Favors innovation over regulation", "mil_exp": 3.4, "internet": 92.0, "cultural_alignment": "Western"},
    "Japan": {"gdp": 5.1, "influence": 0.88, "position": "Pro-regulation for trust", "mil_exp": 1.0, "internet": 95.0, "cultural_alignment": "Eastern-Western hybrid"},
    "China": {"gdp": 17.7, "influence": 0.93, "position": "Strict state-driven AI governance", "mil_exp": 1.7, "internet": 73.0, "cultural_alignment": "Eastern"},
    "Brazil": {"gdp": 2.0, "influence": 0.75, "position": "Leaning toward EU-style regulation", "mil_exp": 1.4, "internet": 81.0, "cultural_alignment": "Latin American"},
    "India": {"gdp": 3.7, "influence": 0.82, "position": "Strategic tech balancing", "mil_exp": 2.4, "internet": 43.0, "cultural_alignment": "South Asian"},
    "Russia": {"gdp": 1.8, "influence": 0.78, "position": "Sovereign tech control", "mil_exp": 4.3, "internet": 85.0, "cultural_alignment": "Eastern"},
    "Iraq": {"gdp": 0.2, "influence": 0.42, "position": "Developing governance framework", "mil_exp": 3.5, "internet": 49.0, "cultural_alignment": "Middle East"},
    "Qatar": {"gdp": 0.18, "influence": 0.68, "position": "Tech-forward with state oversight", "mil_exp": 3.7, "internet": 99.0, "cultural_alignment": "Middle East"},
    "NATO": {"gdp": 25.0, "influence": 0.97, "position": "Collective security & data interoperability", "mil_exp": 2.5, "internet": 90.0, "cultural_alignment": "Western Alliance"},
    "Greenland": {"gdp": 0.003, "influence": 0.45, "position": "Emerging Arctic tech governance", "mil_exp": 0.0, "internet": 68.0, "cultural_alignment": "Nordic"},
    "Venezuela": {"gdp": 0.048, "influence": 0.58, "position": "State-controlled digital infrastructure", "mil_exp": 0.9, "internet": 72.0, "cultural_alignment": "Latin American"}
}

EXTERNAL_INFLUENCES = {
    "GDPR": {"type": "regulation", "strength": 0.9, "targets": ["European Union", "United Kingdom", "Brazil"]},
    "US Export Controls": {"type": "policy", "strength": 0.85, "targets": ["China", "Russia", "Iraq"]},
    "Belt & Road Initiative": {"type": "economic", "strength": 0.75, "targets": ["Brazil", "Qatar", "Dubai"]},
    "AUKUS Agreement": {"type": "alliance", "strength": 0.8, "targets": ["United States", "United Kingdom", "Japan"]},
    "UN AI Ethics": {"type": "norm", "strength": 0.6, "targets": ["India", "Brazil", "NATO"]}
}

INTERNAL_INFLUENCES = {
    "Democratic Norms": {"strength": 0.85, "countries": ["United States", "European Union", "United Kingdom", "Japan", "India", "Brazil"]},
    "Tech Nationalism": {"strength": 0.9, "countries": ["China", "Russia", "United States"]},
    "Post-Colonial Sovereignty": {"strength": 0.7, "countries": ["India", "Brazil", "Iraq", "Qatar"]},
    "Energy Wealth": {"strength": 0.75, "countries": ["Russia", "Qatar", "Dubai", "Venezuela"]},
    "Military-Tech Integration": {"strength": 0.8, "countries": ["United States", "China", "Russia", "NATO"]}
}

# =============================================================================
# EMBED THE 3D ANIMATED VISUALIZATION
# =============================================================================

html_code = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <style>
        * { margin: 0; padding: 0; box-sizing: border-box; }
        body {
            font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', sans-serif;
            background: #0a0e27;
            color: #e0e7ff;
        }
        .container {
            display: grid;
            grid-template-columns: 300px 1fr;
            gap: 1rem;
            padding: 1rem;
            height: 100vh;
        }
        .controls {
            background: rgba(26, 31, 58, 0.8);
            border: 1px solid rgba(102, 126, 234, 0.2);
            border-radius: 12px;
            padding: 1rem;
            overflow-y: auto;
        }
        .control-section {
            margin-bottom: 1.5rem;
            padding-bottom: 1rem;
            border-bottom: 1px solid rgba(102, 126, 234, 0.1);
        }
        .control-section:last-child { border-bottom: none; }
        .control-section h3 {
            font-size: 0.85rem;
            text-transform: uppercase;
            letter-spacing: 0.05em;
            color: #818cf8;
            margin-bottom: 0.75rem;
        }
        .control-group {
            margin-bottom: 1rem;
        }
        .control-group label {
            display: flex;
            justify-content: space-between;
            font-size: 0.85rem;
            margin-bottom: 0.5rem;
            color: #c7d2fe;
        }
        .value {
            color: #818cf8;
            font-weight: 600;
        }
        .slider {
            width: 100%;
            height: 6px;
            border-radius: 3px;
            background: linear-gradient(90deg, #4c1d95 0%, #818cf8 100%);
            outline: none;
            -webkit-appearance: none;
        }
        .slider::-webkit-slider-thumb {
            -webkit-appearance: none;
            appearance: none;
            width: 16px;
            height: 16px;
            border-radius: 50%;
            background: #818cf8;
            cursor: pointer;
            box-shadow: 0 0 10px rgba(129, 140, 248, 0.5);
        }
        .button {
            width: 100%;
            padding: 0.75rem;
            margin-bottom: 0.5rem;
            border: none;
            border-radius: 8px;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            font-weight: 600;
            font-size: 0.9rem;
            cursor: pointer;
            transition: transform 0.2s, box-shadow 0.2s;
        }
        .button:hover {
            transform: translateY(-2px);
            box-shadow: 0 4px 12px rgba(102, 126, 234, 0.4);
        }
        .button.secondary {
            background: rgba(129, 140, 248, 0.2);
            border: 1px solid #818cf8;
        }
        .viz-container {
            position: relative;
            background: rgba(10, 14, 39, 0.6);
            border: 1px solid rgba(102, 126, 234, 0.2);
            border-radius: 12px;
            overflow: hidden;
        }
        #canvas {
            width: 100%;
            height: 100%;
            display: block;
        }
        .stats {
            position: absolute;
            top: 1rem;
            right: 1rem;
            display: grid;
            grid-template-columns: repeat(2, 1fr);
            gap: 0.75rem;
        }
        .stat {
            background: rgba(26, 31, 58, 0.9);
            padding: 0.75rem;
            border-radius: 8px;
            border: 1px solid rgba(102, 126, 234, 0.2);
            backdrop-filter: blur(10px);
        }
        .stat-label {
            display: block;
            font-size: 0.7rem;
            text-transform: uppercase;
            letter-spacing: 0.05em;
            color: #94a3b8;
            margin-bottom: 0.25rem;
        }
        .stat-value {
            display: block;
            font-size: 1.25rem;
            font-weight: 700;
            color: #818cf8;
        }
        .legend {
            position: absolute;
            bottom: 1rem;
            left: 1rem;
            background: rgba(26, 31, 58, 0.9);
            padding: 1rem;
            border-radius: 8px;
            border: 1px solid rgba(102, 126, 234, 0.2);
            backdrop-filter: blur(10px);
        }
        .legend h4 {
            font-size: 0.85rem;
            margin-bottom: 0.5rem;
            color: #818cf8;
        }
        .legend-item {
            display: flex;
            align-items: center;
            margin-bottom: 0.4rem;
            font-size: 0.75rem;
        }
        .legend-color {
            width: 12px;
            height: 12px;
            border-radius: 50%;
            margin-right: 0.5rem;
        }
        .toggle-group {
            display: grid;
            gap: 0.5rem;
        }
        .toggle-btn {
            padding: 0.6rem;
            border: 1px solid rgba(129, 140, 248, 0.3);
            border-radius: 6px;
            background: rgba(129, 140, 248, 0.1);
            color: #c7d2fe;
            font-size: 0.85rem;
            cursor: pointer;
            transition: all 0.2s;
        }
        .toggle-btn.active {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            border-color: transparent;
        }
        .toggle-btn:hover {
            border-color: #818cf8;
        }
    </style>
</head>
<body>
    <div class="container">
        <div class="controls">
            <div class="control-section">
                <h3>Influence Parameters</h3>
                <div class="control-group">
                    <label>External Pressure <span class="value" id="ext_val">70%</span></label>
                    <input type="range" class="slider" id="external_pressure" min="0" max="100" value="70" oninput="updateParams()">
                </div>
                <div class="control-group">
                    <label>Internal Forces <span class="value" id="int_val">60%</span></label>
                    <input type="range" class="slider" id="internal_forces" min="0" max="100" value="60" oninput="updateParams()">
                </div>
                <div class="control-group">
                    <label>Policy Convergence <span class="value" id="conv_val">45%</span></label>
                    <input type="range" class="slider" id="convergence" min="0" max="100" value="45" oninput="updateParams()">
                </div>
                <div class="control-group">
                    <label>Time Evolution <span class="value" id="time_val">0 mo</span></label>
                    <input type="range" class="slider" id="time" min="0" max="36" value="0" oninput="updateParams()">
                </div>
            </div>

            <div class="control-section">
                <h3>Visualization Options</h3>
                <div class="toggle-group">
                    <button class="toggle-btn active" id="show_arrows" onclick="toggleArrows()">🔵 Policy Pressures</button>
                    <button class="toggle-btn active" id="show_spheres" onclick="toggleSpheres()">🟣 Cultural Forces</button>
                    <button class="toggle-btn active" id="show_connections" onclick="toggleConnections()">🔗 Alignments</button>
                    <button class="toggle-btn active" id="show_labels" onclick="toggleLabels()">🏷️ Labels</button>
                </div>
            </div>

            <div class="control-section">
                <h3>Animation</h3>
                <button class="button" onclick="toggleAnimation()"><span id="anim_text">▶ Start Animation</span></button>
                <button class="button secondary" onclick="resetView()">↺ Reset View</button>
            </div>

            <div class="control-section">
                <h3>Information</h3>
                <div style="font-size: 0.75rem; color: #94a3b8; line-height: 1.5;">
                    <p style="margin-bottom: 0.5rem;"><strong>Click & Drag:</strong> Rotate view</p>
                    <p style="margin-bottom: 0.5rem;"><strong>Scroll:</strong> Zoom</p>
                    <p style="margin-bottom: 0.5rem;"><strong>Hover:</strong> View details</p>
                </div>
            </div>
        </div>

        <div class="viz-container">
            <canvas id="canvas"></canvas>
            <div class="stats">
                <div class="stat">
                    <span class="stat-label">Countries</span>
                    <span class="stat-value">14</span>
                </div>
                <div class="stat">
                    <span class="stat-label">Influences</span>
                    <span class="stat-value" id="influence_count">10</span>
                </div>
                <div class="stat">
                    <span class="stat-label">Alignment</span>
                    <span class="stat-value" id="alignment_score">45%</span>
                </div>
                <div class="stat">
                    <span class="stat-label">Clusters</span>
                    <span class="stat-value" id="cluster_count">4</span>
                </div>
            </div>

            <div class="legend">
                <h4>Legend</h4>
                <div class="legend-item">
                    <div class="legend-color" style="background: #667eea;"></div>
                    <span>Western-aligned</span>
                </div>
                <div class="legend-item">
                    <div class="legend-color" style="background: #ef4444;"></div>
                    <span>State-controlled</span>
                </div>
                <div class="legend-item">
                    <div class="legend-color" style="background: #f59e0b;"></div>
                    <span>Hybrid approach</span>
                </div>
                <div class="legend-item">
                    <div class="legend-color" style="background: #10b981;"></div>
                    <span>Regional/Developing</span>
                </div>
            </div>
        </div>
    </div>

    <script src="https://cdnjs.cloudflare.com/ajax/libs/three.js/r128/three.min.js"></script>
    <script>
        // Data structures
        const countries = [
            {id:'US',gdp:21.0,inf:0.95,pos:[0.8,0.9,0.7],c:0x667eea,align:'Western'},
            {id:'EU',gdp:15.0,inf:0.90,pos:[0.9,0.85,0.9],c:0x667eea,align:'Western'},
            {id:'CN',gdp:17.7,inf:0.93,pos:[0.2,0.9,0.3],c:0xef4444,align:'State'},
            {id:'UK',gdp:3.2,inf:0.85,pos:[0.85,0.8,0.85],c:0x667eea,align:'Western'},
            {id:'JP',gdp:5.1,inf:0.88,pos:[0.7,0.85,0.8],c:0xf59e0b,align:'Hybrid'},
            {id:'IN',gdp:3.7,inf:0.82,pos:[0.6,0.8,0.6],c:0xf59e0b,align:'Hybrid'},
            {id:'BR',gdp:2.0,inf:0.75,pos:[0.7,0.7,0.65],c:0x10b981,align:'Regional'},
            {id:'RU',gdp:1.8,inf:0.78,pos:[0.3,0.75,0.4],c:0xef4444,align:'State'},
            {id:'NATO',gdp:25.0,inf:0.97,pos:[0.85,0.95,0.85],c:0x667eea,align:'Western'},
            {id:'Dubai',gdp:0.5,inf:0.7,pos:[0.65,0.65,0.6],c:0xf59e0b,align:'Hybrid'},
            {id:'Qatar',gdp:0.18,inf:0.68,pos:[0.6,0.65,0.55],c:0x10b981,align:'Regional'},
            {id:'Iraq',gdp:0.2,inf:0.42,pos:[0.4,0.4,0.45],c:0x10b981,align:'Regional'},
            {id:'Greenland',gdp:0.003,inf:0.45,pos:[0.5,0.4,0.5],c:0x10b981,align:'Regional'},
            {id:'Venezuela',gdp:0.048,inf:0.58,pos:[0.35,0.55,0.4],c:0xef4444,align:'State'}
        ];

        const policyArrows = [
            {name:'GDPR',targets:['EU','UK','BR'],strength:0.9,color:0x667eea},
            {name:'US Export',targets:['CN','RU','Iraq'],strength:0.85,color:0xef4444},
            {name:'Belt&Road',targets:['BR','Qatar','Dubai'],strength:0.75,color:0xfbbf24},
            {name:'AUKUS',targets:['US','UK','JP'],strength:0.8,color:0x667eea},
            {name:'UN Ethics',targets:['IN','BR','NATO'],strength:0.6,color:0x10b981}
        ];

        const culturalSpheres = [
            {name:'Democratic',countries:['US','EU','UK','JP','IN','BR'],strength:0.85,color:0x667eea},
            {name:'TechNat',countries:['CN','RU','US'],strength:0.9,color:0xef4444},
            {name:'PostColonial',countries:['IN','BR','Iraq','Qatar'],strength:0.7,color:0x10b981},
            {name:'Energy',countries:['RU','Qatar','Dubai','Venezuela'],strength:0.75,color:0xfbbf24},
            {name:'MilTech',countries:['US','CN','RU','NATO'],strength:0.8,color:0xa855f7}
        ];

        // Scene setup
        let scene, camera, renderer, controls;
        let nodes = [], arrows = [], spheres = [], connections = [], labels = [];
        let isAnimating = false;
        let animationFrame = 0;
        let showArrows = true, showSpheres = true, showConnections = true, showLabels = true;

        function init() {
            const canvas = document.getElementById('canvas');
            const w = canvas.clientWidth, h = canvas.clientHeight;

            // Scene
            scene = new THREE.Scene();
            scene.background = new THREE.Color(0x0a0e27);

            // Camera
            camera = new THREE.PerspectiveCamera(60, w/h, 0.1, 1000);
            camera.position.set(25, 20, 25);
            camera.lookAt(0, 0, 0);

            // Renderer
            renderer = new THREE.WebGLRenderer({canvas, antialias: true});
            renderer.setSize(w, h);
            renderer.setPixelRatio(window.devicePixelRatio);

            // Lights
            const ambientLight = new THREE.AmbientLight(0xffffff, 0.5);
            scene.add(ambientLight);

            const pointLight1 = new THREE.PointLight(0x667eea, 1, 100);
            pointLight1.position.set(20, 20, 20);
            scene.add(pointLight1);

            const pointLight2 = new THREE.PointLight(0x764ba2, 0.8, 100);
            pointLight2.position.set(-20, -20, -20);
            scene.add(pointLight2);

            // Create nodes
            createNodes();
            createPolicyArrows();
            createCulturalSpheres();
            createConnections();

            // Mouse controls
            let isDragging = false, previousMousePosition = {x: 0, y: 0};

            canvas.addEventListener('mousedown', (e) => {
                isDragging = true;
                previousMousePosition = {x: e.clientX, y: e.clientY};
            });

            canvas.addEventListener('mousemove', (e) => {
                if (isDragging) {
                    const deltaX = e.clientX - previousMousePosition.x;
                    const deltaY = e.clientY - previousMousePosition.y;

                    const rotationSpeed = 0.005;
                    camera.position.applyAxisAngle(new THREE.Vector3(0, 1, 0), deltaX * rotationSpeed);

                    const horizontalAxis = new THREE.Vector3(1, 0, 0);
                    camera.position.applyAxisAngle(horizontalAxis, deltaY * rotationSpeed);

                    camera.lookAt(0, 0, 0);
                    previousMousePosition = {x: e.clientX, y: e.clientY};
                }
            });

            canvas.addEventListener('mouseup', () => { isDragging = false; });
            canvas.addEventListener('mouseleave', () => { isDragging = false; });

            canvas.addEventListener('wheel', (e) => {
                e.preventDefault();
                const zoomSpeed = 0.001;
                const direction = e.deltaY > 0 ? 1 : -1;
                camera.position.multiplyScalar(1 + direction * zoomSpeed * Math.abs(e.deltaY));
            });

            // Start animation loop
            animate();
        }

        function createNodes() {
            countries.forEach((country, i) => {
                const size = 0.3 + (country.inf * 0.5);
                const geometry = new THREE.SphereGeometry(size, 32, 32);
                const material = new THREE.MeshPhongMaterial({
                    color: country.c,
                    emissive: country.c,
                    emissiveIntensity: 0.3,
                    shininess: 30
                });
                const mesh = new THREE.Mesh(geometry, material);

                // Position in 3D space
                const x = (country.pos[0] - 0.5) * 30;
                const y = (country.pos[1] - 0.5) * 25;
                const z = (country.pos[2] - 0.5) * 30;
                mesh.position.set(x, y, z);

                scene.add(mesh);

                // Create label
                const label = createLabel(country.id);
                label.position.set(x, y + size + 0.8, z);
                scene.add(label);

                nodes.push({mesh, label, country, initialPos: mesh.position.clone()});
                labels.push(label);
            });
        }

        function createLabel(text) {
            const canvas = document.createElement('canvas');
            const context = canvas.getContext('2d');
            canvas.width = 256;
            canvas.height = 128;

            context.fillStyle = '#ffffff';
            context.font = 'Bold 48px Arial';
            context.textAlign = 'center';
            context.fillText(text, 128, 80);

            const texture = new THREE.Texture(canvas);
            texture.needsUpdate = true;

            const material = new THREE.SpriteMaterial({map: texture});
            const sprite = new THREE.Sprite(material);
            sprite.scale.set(3, 1.5, 1);

            return sprite;
        }

        function createPolicyArrows() {
            policyArrows.forEach(policy => {
                policy.targets.forEach(targetId => {
                    const targetNode = nodes.find(n => n.country.id === targetId);
                    if (!targetNode) return;

                    const origin = new THREE.Vector3(0, 10, 0);
                    const direction = new THREE.Vector3().subVectors(targetNode.mesh.position, origin);
                    const length = direction.length();

                    const arrowHelper = new THREE.ArrowHelper(
                        direction.normalize(),
                        origin,
                        length,
                        policy.color,
                        length * 0.2,
                        length * 0.1
                    );

                    arrowHelper.userData = {type: 'arrow', policy};
                    scene.add(arrowHelper);
                    arrows.push(arrowHelper);
                });
            });
        }

        function createCulturalSpheres() {
            culturalSpheres.forEach(sphere => {
                sphere.countries.forEach(countryId => {
                    const node = nodes.find(n => n.country.id === countryId);
                    if (!node) return;

                    const geometry = new THREE.SphereGeometry(2.5, 32, 32);
                    const material = new THREE.MeshBasicMaterial({
                        color: sphere.color,
                        transparent: true,
                        opacity: 0.1,
                        wireframe: true
                    });
                    const mesh = new THREE.Mesh(geometry, material);
                    mesh.position.copy(node.mesh.position);
                    mesh.userData = {type: 'cultural', sphere};

                    scene.add(mesh);
                    spheres.push(mesh);
                });
            });
        }

        function createConnections() {
            for (let i = 0; i < nodes.length; i++) {
                for (let j = i + 1; j < nodes.length; j++) {
                    const dist = nodes[i].mesh.position.distanceTo(nodes[j].mesh.position);
                    if (dist < 12) {
                        const material = new THREE.LineBasicMaterial({
                            color: 0x667eea,
                            transparent: true,
                            opacity: Math.max(0.1, 1 - dist/15)
                        });
                        const geometry = new THREE.BufferGeometry().setFromPoints([
                            nodes[i].mesh.position,
                            nodes[j].mesh.position
                        ]);
                        const line = new THREE.Line(geometry, material);
                        scene.add(line);
                        connections.push(line);
                    }
                }
            }
        }

        function updateParams() {
            const extPressure = parseInt(document.getElementById('external_pressure').value) / 100;
            const intForces = parseInt(document.getElementById('internal_forces').value) / 100;
            const convergence = parseInt(document.getElementById('convergence').value) / 100;
            const time = parseInt(document.getElementById('time').value);

            document.getElementById('ext_val').textContent = Math.round(extPressure * 100) + '%';
            document.getElementById('int_val').textContent = Math.round(intForces * 100) + '%';
            document.getElementById('conv_val').textContent = Math.round(convergence * 100) + '%';
            document.getElementById('time_val').textContent = time + ' mo';

            // Update node positions based on convergence
            const center = new THREE.Vector3(0, 0, 0);
            nodes.forEach(node => {
                const targetPos = node.initialPos.clone().lerp(center, convergence * 0.7);
                node.mesh.position.lerp(targetPos, 0.1);
                node.label.position.copy(node.mesh.position);
                node.label.position.y += 1.2;
            });

            // Update stats
            const alignment = Math.round(convergence * 100);
            document.getElementById('alignment_score').textContent = alignment + '%';
            document.getElementById('cluster_count').textContent = Math.max(1, Math.round(5 * (1 - convergence)));

            // Update arrow visibility based on external pressure
            arrows.forEach(arrow => {
                arrow.visible = showArrows;
                arrow.material.opacity = extPressure * 0.8;
            });

            // Update sphere visibility based on internal forces
            spheres.forEach(sphere => {
                sphere.visible = showSpheres;
                sphere.material.opacity = intForces * 0.15;
            });

            // Update connections
            updateConnections();
        }

        function updateConnections() {
            connections.forEach(conn => scene.remove(conn));
            connections = [];
            createConnections();
            connections.forEach(conn => conn.visible = showConnections);
        }

        function toggleArrows() {
            showArrows = !showArrows;
            arrows.forEach(arrow => arrow.visible = showArrows);
            document.getElementById('show_arrows').classList.toggle('active');
        }

        function toggleSpheres() {
            showSpheres = !showSpheres;
            spheres.forEach(sphere => sphere.visible = showSpheres);
            document.getElementById('show_spheres').classList.toggle('active');
        }

        function toggleConnections() {
            showConnections = !showConnections;
            connections.forEach(conn => conn.visible = showConnections);
            document.getElementById('show_connections').classList.toggle('active');
        }

        function toggleLabels() {
            showLabels = !showLabels;
            labels.forEach(label => label.visible = showLabels);
            document.getElementById('show_labels').classList.toggle('active');
        }

        function toggleAnimation() {
            isAnimating = !isAnimating;
            document.getElementById('anim_text').textContent = isAnimating ? '⏸ Pause' : '▶ Start Animation';
            if (isAnimating) runAnimation();
        }

        function runAnimation() {
            if (!isAnimating) return;

            const timeSlider = document.getElementById('time');
            const convSlider = document.getElementById('convergence');
            let timeVal = parseInt(timeSlider.value);
            let convVal = parseInt(convSlider.value);

            if (timeVal < 36) {
                timeVal += 0.5;
                timeSlider.value = timeVal;

                // Gradually increase convergence
                if (convVal < 90) {
                    convVal = Math.min(90, convVal + 1);
                    convSlider.value = convVal;
                }

                updateParams();
                setTimeout(runAnimation, 100);
            } else {
                isAnimating = false;
                document.getElementById('anim_text').textContent = '▶ Start Animation';
            }
        }

        function resetView() {
            camera.position.set(25, 20, 25);
            camera.lookAt(0, 0, 0);
            document.getElementById('external_pressure').value = 70;
            document.getElementById('internal_forces').value = 60;
            document.getElementById('convergence').value = 45;
            document.getElementById('time').value = 0;
            updateParams();
        }

        function animate() {
            requestAnimationFrame(animate);

            // Gentle auto-rotation when not animating
            if (!isAnimating) {
                const time = Date.now() * 0.0001;
                camera.position.x = Math.sin(time) * 30;
                camera.position.z = Math.cos(time) * 30;
                camera.lookAt(0, 0, 0);
            }

            renderer.render(scene, camera);
        }

        // Initialize on load
        window.addEventListener('load', init);
        window.addEventListener('resize', () => {
            const canvas = document.getElementById('canvas');
            camera.aspect = canvas.clientWidth / canvas.clientHeight;
            camera.updateProjectionMatrix();
            renderer.setSize(canvas.clientWidth, canvas.clientHeight);
        });
    </script>
</body>
</html>
"""

# Display the visualization
components.html(html_code, height=800, scrolling=False)

# =============================================================================
# INFLUENCE ANALYSIS SECTION
# =============================================================================

st.markdown("---")
st.header("📊 Influence Analysis")

with st.expander("📖 Understanding the Influence Map", expanded=False):
    st.markdown("""
    ### How to Read This Visualization

    **External Policy Influences** (Blue Arrows):
    - **GDPR** → Drives strict data protection standards
    - **US Export Controls** → Restricts technology transfers
    - **Belt & Road Initiative** → Creates economic dependencies
    - **AUKUS Agreement** → Strengthens defense tech cooperation
    - **UN AI Ethics** → Promotes global norms

    **Internal Cultural Forces** (Purple Spheres):
    - **Democratic Norms** → Values transparency and rights
    - **Tech Nationalism** → Prioritizes domestic control
    - **Post-Colonial Sovereignty** → Resists external pressure
    - **Energy Wealth** → Uses resources for leverage
    - **Military-Tech Integration** → Fuses defense and AI

    **Interpretation:**
    - **Tight clusters** → Similar policy approaches
    - **Isolated nodes** → Unique governance stance
    - **Many influences** → Higher pressure to conform
    - **Few influences** → Greater policy autonomy

    **Animation shows:**
    - How policies converge over time
    - Impact of increasing coordination pressure
    - Effect of trust networks on alignment
    """)

# Create influence summary table
st.subheader("🔬 Influence Summary by Country")

influence_data = []
for country_name, data in default_data.items():
    ext_count = sum(1 for p in EXTERNAL_INFLUENCES.values() if country_name in p["targets"])
    int_count = sum(1 for f in INTERNAL_INFLUENCES.values() if country_name in f["countries"])

    influence_data.append({
        "Country": country_name,
        "GDP (T USD)": f"${data['gdp']:.2f}",
        "Influence": data["influence"],
        "External": ext_count,
        "Internal": int_count,
        "Total": ext_count + int_count,
        "Alignment": data["cultural_alignment"]
    })

df_influence = pd.DataFrame(influence_data).sort_values("Total", ascending=False)
st.dataframe(df_influence, use_container_width=True, hide_index=True)

# Download option
st.download_button(
    label="📥 Download Influence Data",
    data=df_influence.to_csv(index=False),
    file_name="influence_map_data.csv",
    mime="text/csv"
)

st.markdown("---")
st.info("💡 **Tip**: Use the animation to simulate how policies might converge under different coordination scenarios.")
'''

with open('pages/20_3D_Influence_Map.py', 'w') as f:
    f.write(influence_map_code)

print("✅ 3D Influence Map page created")

✅ 3D Influence Map page created


In [5]:

# === Launch Auracelle Charlie - Live 2025 Simulation ===
import subprocess
from pyngrok import ngrok
import time

# Kill old tunnels and servers
!pkill -f ngrok || true
!pkill -f streamlit || true
try:
    ngrok.kill()
    ngrok.disconnect()
except:
    pass

# Start Streamlit
process = subprocess.Popen(["streamlit", "run", "app.py", "--server.port=8501", "--server.address=0.0.0.0"])

# Start Ngrok with reserved domain
public_url = ngrok.connect(8501, hostname="aiwargame.ngrok.app")
print("✅ Tunnel created:", public_url)

from IPython.display import HTML, display
display(HTML(f'<a href="{public_url}" target="_blank" style="font-size:20px; font-weight:bold; color:#0066cc;">🔗 Launch Auracelle Charlie - Live 2025</a>'))


^C
^C
✅ Tunnel created: NgrokTunnel: "https://aiwargame.ngrok.app" -> "http://localhost:8501"


In [6]:

# --- Add "Agentic AI Demo" Streamlit page ---
# This cell creates pages/90_Agentic_AI_Demo.py so it shows up as a separate page in the app.
# It is self-contained and will not modify your existing Simulation page or UI conventions.

import os
os.makedirs("pages", exist_ok=True)

agentic_page = r"""
# SPDX-License-Identifier: MIT
import time
import math
import random
from dataclasses import dataclass, field
from typing import List, Dict, Tuple

import numpy as np
import pandas as pd
import streamlit as st

st.set_page_config(page_title="Agentic AI Demo", layout="wide")

# ------------------------------
# Minimal "aiwargame" sample env
# ------------------------------
# This is a lightweight, self-contained demonstration so it won't clash with your main Simulation logic.
# If your main app exposes shared policy/state helpers later, you can import them here and bypass this local demo.

@dataclass
class Actor:
    name: str
    influence: float = 1.0          # baseline influence (0-3 nominal range)
    compliance: float = 0.5         # nominal "policy compliance / safety posture" (0-1)
    payoff: float = 0.0             # accumulated reward

@dataclass
class Env:
    actors: Dict[str, Actor] = field(default_factory=dict)
    step_count: int = 0
    max_steps: int = 8
    history: List[Dict] = field(default_factory=list)

    def reset(self):
        self.step_count = 0
        self.history = []
        # lightweight baseline
        self.actors = {
            "US": Actor("US", influence=1.2, compliance=0.55),
            "EU": Actor("EU", influence=1.1, compliance=0.65),
            "China": Actor("China", influence=1.25, compliance=0.45),
        }
        return self._obs()

    def _obs(self):
        # Observation is a dict of actor states plus a derived "risk" score
        state = {
            k: {"influence": v.influence, "compliance": v.compliance, "payoff": v.payoff}
            for k, v in self.actors.items()
        }
        # simple aggregate "systemic risk" proxy
        mean_comp = np.mean([v.compliance for v in self.actors.values()])
        mean_inf = np.mean([v.influence for v in self.actors.values()])
        risk = float(max(0.0, 1.2 - (0.6*mean_comp + 0.2*min(1.5, mean_inf))))
        state["_derived"] = {"risk": risk, "t": self.step_count}
        return state

    def step(self, action: str, controlled_actor: str = "US"):
        # Apply an action taken by the "agent" controlling the selected actor.
        # Actions trade off compliance, influence, and a small coalition effect.
        a = self.actors[controlled_actor]

        # stochasticity for realism
        jitter = lambda s: s + random.uniform(-0.01, 0.01)

        coalition_bonus = 0.0
        if action == "Export Controls":
            a.influence = max(0.6, jitter(a.influence + 0.02))
            a.compliance = min(1.0, jitter(a.compliance + 0.04))
        elif action == "Safety Audits":
            a.influence = max(0.6, jitter(a.influence - 0.01))
            a.compliance = min(1.0, jitter(a.compliance + 0.06))
            coalition_bonus = 0.01
        elif action == "Open Data":
            a.influence = min(1.6, jitter(a.influence + 0.03))
            a.compliance = max(0.1, jitter(a.compliance - 0.02))
        elif action == "Joint Standards":
            a.influence = min(1.6, jitter(a.influence + 0.02))
            a.compliance = min(1.0, jitter(a.compliance + 0.03))
            coalition_bonus = 0.02
        else:
            # No-op / hold
            a.influence = jitter(a.influence)
            a.compliance = jitter(a.compliance)

        # Simple coalition effect: others follow a tiny bit
        for k, other in self.actors.items():
            if k == controlled_actor:
                continue
            other.compliance = float(np.clip(other.compliance + coalition_bonus * random.uniform(0.4, 1.2), 0.1, 1.0))

        # Reward = (own influence * 0.6 + own compliance * 0.8) - systemic risk penalty
        obs = self._obs()
        risk = obs["_derived"]["risk"]
        reward = 0.6*a.influence + 0.8*a.compliance - (0.9 * risk)
        a.payoff += reward

        self.step_count += 1
        done = self.step_count >= self.max_steps

        # Log
        self.history.append({
            "t": self.step_count,
            "actor": controlled_actor,
            "action": action,
            "reward": reward,
            "risk": risk,
            "US_infl": self.actors["US"].influence,
            "US_comp": self.actors["US"].compliance,
            "EU_infl": self.actors["EU"].influence,
            "EU_comp": self.actors["EU"].compliance,
            "CN_infl": self.actors["China"].influence,
            "CN_comp": self.actors["China"].compliance,
        })

        return obs, reward, done, {}

# ------------------------------
# A tiny "Agentic AI" controller
# ------------------------------
# A one-step lookahead agent that evaluates each action with a heuristic objective
# and chooses the best. This is intentionally transparent for demo purposes.

ACTIONS = ["Export Controls", "Safety Audits", "Open Data", "Joint Standards", "Hold/No-Op"]

def evaluate_action(env: Env, action: str, actor: str, sims: int = 12):
    # simulate hypothetical outcome for ranking; keep it light-weight
    # Copy a shallow snapshot for Monte Carlo trials
    scores = []
    for _ in range(sims):
        snap = Env()
        # clone state
        snap.actors = {k: Actor(v.name, v.influence, v.compliance, v.payoff) for k, v in env.actors.items()}
        snap.step_count = env.step_count
        obs, reward, _, _ = snap.step(action, controlled_actor=actor)
        # Heuristic: prefer high own reward, lower systemic risk
        srisk = obs["_derived"]["risk"]
        score = reward - 0.3*srisk
        scores.append(score)
    return float(np.mean(scores))

def agent_choose_action(env: Env, actor: str, stochastic: bool = False):
    candidates = []
    for a in ACTIONS:
        s = evaluate_action(env, a, actor)
        candidates.append((a, s))
    candidates.sort(key=lambda x: x[1], reverse=True)
    if stochastic and random.random() < 0.2:
        return random.choice(ACTIONS)
    return candidates[0][0]

# ------------------------------
# UI
# ------------------------------

st.title("🤖 Agentic AI Demo — Sample Game")
st.caption("A transparent, one-step lookahead agent plays a lightweight version of the AI governance wargame.")

left, right = st.columns([2, 1])

with right:
    st.subheader("Agent Controls")
    controlled_actor = st.selectbox("Controlled Actor", ["US", "EU", "China"], index=0)
    horizon = st.slider("Episode Length (steps)", 4, 16, 8, 1)
    stochastic = st.toggle("Enable stochastic exploration (ε≈0.2)", value=True)
    episodes = st.number_input("Batch episodes", min_value=1, max_value=50, value=1, step=1)
    autoplay = st.toggle("Autoplay (fast)", value=False)
    st.markdown("---")
    st.page_link("app.py", label="⬅ Back to Home", icon="🏠")

with left:
    st.subheader("Run a Sample Game")
    env = Env()
    env.max_steps = horizon

    run_now = st.button("▶ Play Sample Game")

    if run_now:
        obs = env.reset()
        frames = []
        for t in range(horizon):
            action = agent_choose_action(env, controlled_actor, stochastic=stochastic)
            obs, reward, done, _ = env.step(action, controlled_actor)
            row = {"t": t+1, "action": action, "reward": round(reward, 4), "risk": round(obs["_derived"]["risk"], 4)}
            for k, v in env.actors.items():
                row[f"{k}_influence"] = round(v.influence, 3)
                row[f"{k}_compliance"] = round(v.compliance, 3)
                row[f"{k}_payoff"] = round(v.payoff, 3)
            frames.append(row)
            if not autoplay:
                st.write(f"Step {t+1}: **{action}** → reward {row['reward']}, risk {row['risk']}")
                time.sleep(0.25)
            if done:
                break

        df = pd.DataFrame(frames)
        st.markdown("#### Episode Trace")
        st.dataframe(df, use_container_width=True)

        # Simple summaries
        st.markdown("#### Final Payoffs")
        summary = pd.DataFrame([{"Actor": k, "Payoff": round(v.payoff, 4), "Influence": round(v.influence,3), "Compliance": round(v.compliance,3)}
                                for k, v in env.actors.items()]).sort_values("Payoff", ascending=False)
        st.dataframe(summary, use_container_width=True)

    st.markdown("---")
    st.subheader("Batch Evaluation")
    if st.button("🏃 Run Batch"):
        results = []
        for ep in range(episodes):
            env = Env(); env.max_steps = horizon; env.reset()
            for t in range(horizon):
                a = agent_choose_action(env, controlled_actor, stochastic=stochastic)
                env.step(a, controlled_actor)
            results.append({k: v.payoff for k, v in env.actors.items()})
        res_df = pd.DataFrame(results)
        st.markdown("Average payoff over batch:")
        st.dataframe(pd.DataFrame(res_df.mean()).rename(columns={0:"Avg Payoff"}), use_container_width=True)

"""

with open("pages/90_Agentic_AI_Demo.py", "w", encoding="utf-8") as f:
    f.write(agentic_page)

print("✅ Created pages/90_Agentic_AI_Demo.py")


✅ Created pages/90_Agentic_AI_Demo.py


In [7]:

# --- Write auracelle_agent_adapter.py (correctly) ---
adapter_src = r"""
import importlib
import random
from typing import List
import numpy as np
import streamlit as st

def _try_get(module, names: List[str]):
    for n in names:
        if hasattr(module, n):
            return getattr(module, n)
    return None

def _coerce_list(x):
    try:
        return list(x)
    except Exception:
        return None

def load_main_sim_handles():
    try:
        app = importlib.import_module("app")
    except Exception:
        return None
    handles = {}
    handles["policy_options"] = _try_get(app, ["policy_options","POLICY_OPTIONS","policies","POLICY_LIST"])
    handles["policy_effects"] = _try_get(app, ["policy_effect_mappings","policy_effects","POLICY_EFFECTS","policy_effect_map"])
    handles["countries"] = _try_get(app, ["countries","country_list","COUNTRIES","nodes","NODES"])
    handles["roles"] = _try_get(app, ["roles","ROLES"])
    handles["apply_policy"] = _try_get(app, ["apply_policy_effects","apply_policy","apply_effects"])
    handles["get_risk"] = _try_get(app, ["compute_systemic_risk","get_risk","risk_metric"])
    for k in ["policy_options","countries","roles"]:
        if handles.get(k) is not None:
            handles[k] = _coerce_list(handles[k])
    handles["app"] = app
    return handles

def init_state(actors: List[str]):
    if "agent_autoplay_state" not in st.session_state:
        st.session_state.agent_autoplay_state = {
            "t": 0,
            "actors": {a: {"influence": 1.0, "compliance": 0.5, "payoff": 0.0} for a in actors},
            "history": []
        }
    return st.session_state.agent_autoplay_state

def toy_risk(state: dict):
    inf = np.mean([v["influence"] for v in state["actors"].values()])
    comp = np.mean([v["compliance"] for v in state["actors"].values()])
    return float(max(0.0, 1.2 - (0.6*comp + 0.2*min(1.5, inf))))

def derive_risk(handles, state):
    if handles and handles.get("get_risk"):
        try:
            return float(handles["get_risk"](state))
        except Exception:
            pass
    return toy_risk(state)

def step_with_main_effects(handles, state: dict, action: str, controlled_actor: str):
    if handles and handles.get("apply_policy"):
        try:
            new_state = handles["apply_policy"](state, action, controlled_actor)
            st.session_state.agent_autoplay_state = new_state
            risk = derive_risk(handles, new_state)
            a = new_state["actors"][controlled_actor]
            reward = 0.6*a["influence"] + 0.8*a["compliance"] - 0.9*risk
            a["payoff"] += reward
            new_state["t"] = new_state.get("t", 0) + 1
            return reward, risk
        except Exception:
            pass
    pe = None
    if handles and handles.get("policy_effects"):
        pe = handles["policy_effects"]
    if isinstance(pe, dict) and action in pe:
        delta = pe[action]
        a = state["actors"].setdefault(controlled_actor, {"influence": 1.0, "compliance": 0.5, "payoff": 0.0})
        if isinstance(delta, dict):
            if "influence" in delta:
                a["influence"] = float(np.clip(a["influence"] + float(delta["influence"]), 0.1, 2.0))
            if "compliance" in delta:
                a["compliance"] = float(np.clip(a["compliance"] + float(delta["compliance"]), 0.0, 1.0))
        risk = derive_risk(handles, state)
        reward = 0.6*a["influence"] + 0.8*a["compliance"] - 0.9*risk
        a["payoff"] += reward
        state["t"] += 1
        return reward, risk
    # fallback toy step
    a = state["actors"].setdefault(controlled_actor, {"influence": 1.0, "compliance": 0.5, "payoff": 0.0})
    jitter = lambda s: s + random.uniform(-0.01, 0.01)
    if action == "Export Controls":
        a["influence"] = max(0.6, jitter(a["influence"] + 0.02))
        a["compliance"] = min(1.0, jitter(a["compliance"] + 0.04))
    elif action == "Safety Audits":
        a["influence"] = max(0.6, jitter(a["influence"] - 0.01))
        a["compliance"] = min(1.0, jitter(a["compliance"] + 0.06))
    elif action == "Open Data":
        a["influence"] = min(1.6, jitter(a["influence"] + 0.03))
        a["compliance"] = max(0.1, jitter(a["compliance"] - 0.02))
    elif action == "Joint Standards":
        a["influence"] = min(1.6, jitter(a["influence"] + 0.02))
        a["compliance"] = min(1.0, jitter(a["compliance"] + 0.03))
    else:
        a["influence"] = jitter(a["influence"])
        a["compliance"] = jitter(a["compliance"])
    risk = derive_risk(handles, state)
    reward = 0.6*a["influence"] + 0.8*a["compliance"] - 0.9*risk
    a["payoff"] += reward
    state["t"] += 1
    return reward, risk

def get_actions(handles):
    if handles and handles.get("policy_options"):
        return [str(x) for x in handles["policy_options"]]
    return ["Export Controls","Safety Audits","Open Data","Joint Standards","Hold/No-Op"]

def get_actors(handles):
    if handles and handles.get("countries"):
        return [str(x) for x in handles["countries"]]
    return ["US","EU","China"]

def evaluate_action(handles, state, action: str, actor: str, sims: int = 8):
    import copy
    scores = []
    for _ in range(sims):
        s = copy.deepcopy(state)
        reward, risk = step_with_main_effects(handles, s, action, actor)
        scores.append(reward - 0.3*risk)
    return float(np.mean(scores))

def agent_choose_action(handles, state, actor: str, stochastic: bool = False):
    acts = get_actions(handles)
    ranked = [(a, evaluate_action(handles, state, a, actor)) for a in acts]
    ranked.sort(key=lambda x: x[1], reverse=True)
    if stochastic and random.random() < 0.2:
        return random.choice(acts)
    return ranked[0][0]
"""
with open("auracelle_agent_adapter.py", "w", encoding="utf-8") as f:
    f.write(adapter_src)
print("✅ Wrote auracelle_agent_adapter.py")


✅ Wrote auracelle_agent_adapter.py


In [8]:
# --- Write auracelle_agentic_upgrades.py (Agentic AI scaffold integrated) ---
agentic_src = r'''

import random
from typing import Dict, List, Tuple

DEFAULT_POLICY_PACKAGES: List[Dict] = [
    {"name": "Safeguards + Compute Reporting + Audit Corridor",
     "moves": ["Privacy Safeguards", "Compute Reporting", "Cross-Border Audit Pathway"],
     "base_cost": 1.2},
    {"name": "Transparency Exchange + Incident Protocol",
     "moves": ["Transparency Exchange", "Incident Response Protocol"],
     "base_cost": 1.0},
    {"name": "Joint Oversight Board + Redress Mechanism",
     "moves": ["Joint Oversight Board", "User Redress Mechanism"],
     "base_cost": 1.1},
    {"name": "Minimal Deal: Shared Definitions",
     "moves": ["Shared Definitions / Terminology"],
     "base_cost": 0.6},
]

def is_institution_actor(actor: str) -> bool:
    a = (actor or "").strip().lower()
    return any(x in a for x in ["nato", "eu", "oecd", "un", "g7", "g20", "gcc", "mena"])

def infer_opponent_stance(last_moves: List[str]) -> str:
    \"\"\"Classify stance from the last 2–3 moves (conciliatory/neutral/hardline).\"\"\"
    if not last_moves:
        return "neutral"
    recent = [m.lower() for m in last_moves[-3:]]
    hard = sum(any(k in m for k in ["escalat", "sanction", "demand", "ultimatum"]) for m in recent)
    conc = sum(any(k in m for k in ["concess", "offer", "share", "cooperat", "joint"]) for m in recent)
    if hard >= 2 and conc == 0:
        return "hardline"
    if conc >= 2 and hard == 0:
        return "conciliatory"
    return "neutral"

class AutonomousNegotiationAgent:
    \"\"\"Lightweight learning agent (linear Q approximator) with opponent stance in state.\"\"\"

    def __init__(self, name: str, institution: bool = False, alpha: float = 0.15, gamma: float = 0.95, eps: float = 0.15):
        self.name = name
        self.institution = institution
        self.alpha = alpha
        self.gamma = gamma
        self.eps = eps
        # weights[action_name] = dict(feature->weight)
        self.weights: Dict[str, Dict[str, float]] = {}
        self.last_opponent_moves: List[str] = []

    def _features(self, tension: float, stance: str, round_idx: int) -> Dict[str, float]:
        # Tiny, interpretable feature set
        return {
            "bias": 1.0,
            "tension": float(tension),
            "round": float(round_idx),
            "stance_conc": 1.0 if stance == "conciliatory" else 0.0,
            "stance_hard": 1.0 if stance == "hardline" else 0.0,
        }

    def _score(self, action_name: str, feats: Dict[str, float]) -> float:
        w = self.weights.get(action_name)
        if w is None:
            # lazy init
            w = {k: 0.0 for k in feats.keys()}
            self.weights[action_name] = w
        return sum(w.get(k, 0.0) * v for k, v in feats.items())

    def choose_package(self, tension: float, policy_packages: List[Dict], round_idx: int = 0) -> Tuple[Dict, str, float]:
        stance = infer_opponent_stance(self.last_opponent_moves)
        feats = self._features(tension=tension, stance=stance, round_idx=round_idx)

        # epsilon-greedy over packages
        if random.random() < self.eps:
            pkg = random.choice(policy_packages)
        else:
            scored = [(pkg, self._score(pkg["name"], feats)) for pkg in policy_packages]
            scored.sort(key=lambda x: x[1], reverse=True)
            pkg = scored[0][0]

        cost_mult = 1.5 if self.institution else 1.0
        return pkg, stance, cost_mult

    def learn(self, prev_tension: float, new_tension: float, round_idx: int, action_name: str, reward: float):
        stance = infer_opponent_stance(self.last_opponent_moves)
        feats = self._features(tension=prev_tension, stance=stance, round_idx=round_idx)
        q = self._score(action_name, feats)

        # bootstrap target using new tension (proxy for next state)
        next_feats = self._features(tension=new_tension, stance=stance, round_idx=round_idx + 1)
        # optimistic estimate: max over known actions
        if self.weights:
            next_q = max(self._score(a, next_feats) for a in self.weights.keys())
        else:
            next_q = 0.0

        target = reward + self.gamma * next_q
        td = target - q

        w = self.weights[action_name]
        for k, v in feats.items():
            w[k] = w.get(k, 0.0) + self.alpha * td * v

class MediatorAgent:
    \"\"\"Rule/heuristic mediator that proposes 2–3 compromise bundles plus a rationale trace.\"\"\"

    def propose(self, positions: Dict, tension: float, policy_packages: List[Dict]) -> Tuple[List[Dict], Dict]:
        # Simple heuristic: when tension is high, propose lower-cost or legitimacy-building bundles
        sorted_pkgs = sorted(policy_packages, key=lambda p: p.get("base_cost", 1.0))
        if tension >= 0.7:
            picks = [sorted_pkgs[0], sorted_pkgs[1]]
        elif tension >= 0.4:
            picks = [sorted_pkgs[1], sorted_pkgs[2]]
        else:
            picks = [sorted_pkgs[2], sorted_pkgs[3] if len(sorted_pkgs) > 3 else sorted_pkgs[0]]

        rationale = {
            "what_each_side_gains": "Lower immediate escalation risk; clearer pathway to mutual verification and trust-building.",
            "what_each_side_concedes": "Partial flexibility on sovereignty/oversight to gain reciprocal assurances.",
            "predicted_diffusion_impact": "Higher if an institutional actor (EU/NATO/UN/OECD) adopts early; moderate otherwise.",
            "predicted_tension_change": "Decrease expected if accepted; neutral-to-increase if rejected under high tension."
        }
        return picks, rationale

class RedTeamAgent:
    \"\"\"Adversarial stress-tester that selects plausible/worst-timed shocks and scores robustness.\"\"\"

    SHOCKS = [
        "Cross-Border Data Breach",
        "Disinformation Wave",
        "Critical Infrastructure AI Failure",
        "Data Localization Emergency",
        "Supply Chain/Compute Restriction Shock",
    ]

    def select_shock(self, tension: float) -> str:
        # Worst-timed: pick higher-severity shocks when tension is already high
        if tension >= 0.75:
            return "Critical Infrastructure AI Failure"
        if tension >= 0.55:
            return "Cross-Border Data Breach"
        return "Disinformation Wave"

    def score_robustness(self, pre_tension: float, post_tension: float) -> str:
        if post_tension - pre_tension > 0.15:
            return "Fracture risk ↑"
        if pre_tension - post_tension > 0.10:
            return "Adapted"
        return "Held"

'''
with open('auracelle_agentic_upgrades.py','w',encoding='utf-8') as f:
    f.write(agentic_src)
print('✅ Wrote auracelle_agentic_upgrades.py')


✅ Wrote auracelle_agentic_upgrades.py
